<a href="https://colab.research.google.com/github/sixt-manas-choudhary/adani/blob/main/Rail_Ops.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import functions_framework

# Triggered from a message on a Cloud Pub/Sub topic.

import pandas as pd
import numpy as np
from pytz import timezone
from datetime import datetime
from itertools import groupby
from google.cloud import bigquery
from google.cloud import storage
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
import smtplib
from typing import Any

import composer2_airflow_rest_api

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
@functions_framework.cloud_event
def rail_route_optimizer(cloud_event):
    import pandas as pd
    import numpy as np
    client = bigquery.Client(project='apsez-svc-prod-datalake')
    from datetime import datetime
    today = datetime.today().strftime("%Y-%m-%d")
    print("today = ", today)
    now = datetime.now(timezone("Asia/Kolkata")).strftime('%Y-%m-%d %H:%M:%S')
    print("now = ", now)
    today_filename = datetime.today().strftime("%d%m%Y")
    print(today_filename)

    ################################################################################
    # For Distance Data- Loading BigQuery data for Route Details
    ################################################################################

    query_rd="""select coalesce(og_map.fois, origin_code) as origin_code,
    coalesce(to_map.fois, destination_code) destination_code,
    route, rational_distance, critical_non_critical, ss_ds, r_tat_sla, t_tat_sla, closed_rt, active_rt
    from `logistics_cleansed.layer2_route_sla_mst` rsm
    left join (select terminal_code, fois from `logistics_cleansed.layer2_terminal_master`
                 qualify row_number() over(partition by terminal_code)  = 1 ) og_map
    on og_map.terminal_code = rsm.origin_code
    left join (select terminal_code, fois from `logistics_cleansed.layer2_terminal_master`
                 qualify row_number() over(partition by terminal_code)  = 1 ) to_map
    on to_map.terminal_code = rsm.destination_code
    """
    query_job_rd=client.query(query_rd).result()
    route_details = query_job_rd.to_dataframe()

    route_details.drop(columns = ["critical_non_critical","route"], inplace = True)
    route_details = route_details[route_details.origin_code.notnull()]
    route_details = route_details[route_details.destination_code.notnull()]
    route_details.t_tat_sla = route_details.t_tat_sla.fillna(0)
    route_details.closed_rt = route_details.closed_rt.astype(int).round(0)
    route_details.active_rt = route_details.active_rt.astype(int).round(0)

    # t_tat_sla is given at origin
    route_details.rename(columns = {"t_tat_sla":"t_tat_sla_origin"},inplace = True)
    route_details["rational_distance"] = route_details.rational_distance.astype('float64').round(0)
    route_details["t_tat_sla_origin"] = route_details.t_tat_sla_origin.astype('float64').round(0)
    route_details["r_tat_sla"] = route_details.r_tat_sla.astype('float64').round(0)
    # Don't remove t_tat_sla_origin null terminals as they could be base terminals but with 0 pendency

    # Fill null ss_ds with "SS" to avoid any error
    route_details.ss_ds = route_details.ss_ds.fillna("SS")

    route_details = route_details.drop_duplicates(subset = ["origin_code","destination_code"], keep = 'first').reset_index(drop = True)

    # update t_tat_sla for origin
    for i in range(len(route_details)):
      if route_details.loc[i,"rational_distance"] != 0:
        route_details.loc[i,"t_tat_sla_origin"] = 12

    route_details_t_tat = route_details[["origin_code","t_tat_sla_origin"]].drop_duplicates(subset = ["origin_code"], keep = "first").reset_index(drop = True)

    # create t_tat_sla for destination
    route_details.t_tat_sla_destination = float()
    for i in range(len(route_details)):
        for j in range(len(route_details_t_tat)):
            if route_details.loc[i,"destination_code"] == route_details_t_tat.loc[j,"origin_code"]:
                route_details.loc[i,"t_tat_sla_destination"] = float(route_details_t_tat.loc[j,"t_tat_sla_origin"])

    # update t_tat_sla for origin
    for i in range(len(route_details)):
      if route_details.loc[i,"rational_distance"] != 0:
        route_details.loc[i,"t_tat_sla_destination"] = 12

    selected_locations = route_details.reset_index(drop = True)

    for i in range(len(selected_locations)):
        if selected_locations.origin_code[i] == "ALIK":
            selected_locations.t_tat_sla_origin[i] = 15.0
        elif selected_locations.destination_code[i] == "ALIK":
            selected_locations.t_tat_sla_destination[i] = 15.0
        if selected_locations.origin_code[i] == "PPSP":
            selected_locations.t_tat_sla_origin[i] = 12.0
        elif selected_locations.destination_code[i] == "PPSP":
            selected_locations.t_tat_sla_destination[i] = 12.0
        if selected_locations.origin_code[i] == "MDCC":
            selected_locations.t_tat_sla_origin[i] = 18.0
        elif selected_locations.destination_code[i] == "MDCC":
            selected_locations.t_tat_sla_destination[i] = 18.0
        if selected_locations.origin_code[i] == "C":
            selected_locations.t_tat_sla_origin[i] = 28.0
        elif selected_locations.destination_code[i] == "C":
            selected_locations.t_tat_sla_destination[i] = 28.0
        if selected_locations.origin_code[i] == "HAPA":
            selected_locations.t_tat_sla_origin[i] = 12.0
        elif selected_locations.destination_code[i] == "HAPA":
            selected_locations.t_tat_sla_destination[i] = 12.0
        if selected_locations.origin_code[i] == "FL":
            selected_locations.t_tat_sla_origin[i] = 12.0
        elif selected_locations.destination_code[i] == "FL":
            selected_locations.t_tat_sla_destination[i] = 12.0

    for i in range(len(selected_locations)):
      if selected_locations.loc[i,"origin_code"] == "NTSJ":
        selected_locations.loc[i,"origin_code"] = "AFAS"
      if selected_locations.loc[i,"destination_code"] == "NTSJ":
        selected_locations.loc[i,"destination_code"] = "AFAS"

    selected_locations = selected_locations[selected_locations.active_rt == 1]

    distance_data = selected_locations.drop_duplicates(subset = ['origin_code','destination_code'],keep = "first").reset_index(drop = True)
    distance_data_copy = distance_data.copy()

    ##################################################################################
    # For Current Status of the trains- Loading BigQuery data for Train Current Status
    ##################################################################################

    query_t= """select a.*,b.bpc_valid_to,b.remaining_dist,c.base_depot_code, arp.load_plan_teus
    from logistics_semantic.layer4_bt_rake_running_status_snap_hourly_vw as a
    left join logistics_semantic.layer4_rt_all_rake_txr_details_mv as b on (a.rake_name=b.rake_name)
    left join logistics_semantic.layer4_rake_master_vw as c on (a.rake_name=c.rake_name)
    left join (select ob_train_no, load_plan_teus from `logistics_semantic.layer4_rt_all_rail_performance_mv`) arp on arp.ob_train_no = a.ob_train_no"""
    query_job_t = client.query(query_t).result()
    rcs = query_job_t.to_dataframe()
    rcs = rcs[rcs.bu.isin(['ALSPL', 'ALL'])]


    # drop columns that are not required
    rcs.drop(columns = ["actual_running_hrs","departure_delay","examtype","operation_delay_flag",
                        "operation_time","stable_flag","stable_since","sttschngtime","trans_flag",
                        "congestion_alert"],inplace = True)

    # # Uncomment this for all trains
    selected_train_cs = rcs[(rcs.rake_status == 'IN-TRANSIT') | ((rcs.rake_status == 'AT TERMINAL') & (rcs.ob_t.isna()))]

    # # selected trains where wagons is not "-" (Only in-transit vehicles are considered)
    # # selected_train_cs = selected_train_cs[(selected_train_cs.unts != "-")]
    selected_train_cs.unts = selected_train_cs.unts.fillna(45)

    selected_train_cs["eta"] = pd.to_datetime(selected_train_cs["eta"])
    selected_train_cs.rename(columns = {"sttnfrom":"origin_code","sttnto":"destination_code","bpc_valid_to":"TXR_Due_Date","remaining_dist":"TXR_Kms_Remaining","eta":"FOIS_ETA"}, inplace = True)

    selected_train_cs1 = selected_train_cs.astype(object).replace(np.nan, 'None').reset_index(drop = True)

    for i in range(len(selected_train_cs1)):
      if selected_train_cs1.loc[i,"origin_code"] == "NTSJ":
        selected_train_cs1.loc[i,"origin_code"] = "AFAS"
      if selected_train_cs1.loc[i,"destination_code"] == "NTSJ":
        selected_train_cs1.loc[i,"destination_code"] = "AFAS"

    selected_train_cs1 = selected_train_cs1.drop_duplicates(subset = ['origin_code','destination_code'],keep = "first").reset_index(drop = True)

    selected_train_cs1 = selected_train_cs.merge(route_details[["origin_code","destination_code",
                                                                "rational_distance","ss_ds","t_tat_sla_destination"]],
                                                 on = ["origin_code","destination_code"],
                                                 how = "left").reset_index(drop = True)

    selected_train_cs1["t_tat_sla_destination"] = selected_train_cs1["t_tat_sla_destination"].fillna(12)
    selected_train_cs1["t_tat_sla"] = selected_train_cs1["t_tat_sla"].fillna(12)
    selected_train_cs1["unts"] = selected_train_cs1["unts"].fillna(12)
    # selected_train_cs1["ss_ds"] = selected_train_cs1["t_tat_sla"].fillna("SS")

    # for i in range(len(selected_train_cs1)):
    #     if selected_train_cs1.t_tat_sla_destination[i] == "<NA>":
    #         selected_train_cs1.t_tat_sla_destination[i] = 12.0
    # for i in range(len(selected_train_cs1)):
    #     if selected_train_cs1.t_tat_sla[i] == "<NA>":
    #         selected_train_cs1.t_tat_sla[i] = 12.0

    selected_train_cs1['t_tat_sla_destination'] = selected_train_cs1['t_tat_sla_destination'].astype('float64').round(0)
    selected_train_cs1['rational_distance'] = selected_train_cs1['rational_distance'].astype('float64').round(0)
    selected_train_cs1['t_tat_sla'] = selected_train_cs1['t_tat_sla'].astype('float64').round(0)
    selected_train_cs1['transit_sla'] = selected_train_cs1['transit_sla'].astype('float64').round(0)
    selected_train_cs1['TXR_Due_Date'] = pd.to_datetime(selected_train_cs1['TXR_Due_Date']).dt.date


    selected_train_cs1.drop(columns = ["address","bu","cmdt","cnsg","cnsr","crntdvsn","crntdvsn",
                                       "destination_name",'fnr','fois_location', 'gps_tracker_flag', 'ist_timestamp', 'latitude',
                                       'leflag', 'loadid', 'loadstts', 'loadtype', 'loco', 'longitude', 'ob_r',
                                       'ob_t', 'ob_train_no', 'origin_name','sequence','rnk','remaining_kms',
                                       'crntsttn','remaining_kms','rnk','source','running_delay',
                                       'trans_flag_calc','refg_flag_calc','ref_flag','rakeid','rakeownr'], inplace = True)

    # selected trains next available time (FOIS ETA + operation time at destination) or (+ 6 days)
    # our algorithm is supposed to schedule its txr trip by itself
    selected_train_cs1["t_tat_sla_destination"] = selected_train_cs1["t_tat_sla_destination"].fillna(0)
    # selected_train_cs1["next_availability"] = pd.to_datetime(selected_train_cs1["FOIS_ETA"]) + pd.to_timedelta(selected_train_cs1["t_tat_sla_destination"], unit = 'h' )
    # next_availability = FOIS ETA

    # If a train doesn't have an arrival date then the train must not be scheduled for it's next trip
    # selected_train_cs1 = selected_train_cs1[~((selected_train_cs1["FOIS_ETA"].isna()))]
    selected_train_cs1.FOIS_ETA = selected_train_cs1.FOIS_ETA.fillna(now)


    selected_train_cs1["FOIS_ETA"] = pd.to_datetime(selected_train_cs1["FOIS_ETA"]).apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
    selected_train_cs1["next_availability"] = pd.to_datetime(selected_train_cs1["FOIS_ETA"])
    selected_train_cs1["next_availability"] = pd.to_datetime(selected_train_cs1["next_availability"]).apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
    selected_train_cs1["next_available_date"] = pd.to_datetime(selected_train_cs1["next_availability"]).apply(lambda x: x.strftime('%Y-%m-%d'))

    # # if the train is at base depot then next availability is FOIS ETA + 6 more days (as train takes time for maintenance)
    # for i in range(len(selected_train_cs1)):
    #     if selected_train_cs1.destination_code[i] == selected_train_cs1.base_depot_code[i]:
    #       if pd.to_datetime(selected_train_cs1["TXR_Due_Date"][i]).date() < pd.to_datetime(selected_train_cs1["FOIS_ETA"][i]).date():
    #         selected_train_cs1.next_availability[i] = pd.to_datetime(selected_train_cs1["FOIS_ETA"][i]) + pd.to_timedelta(6, unit = 'd' )

    # only select trains which are in-transit within 24 hours from now.
    selected_train_cs2 = selected_train_cs1[(
                                              # (selected_train_cs1["rake_status"] == "In-Transit")
                                              # & (pd.to_datetime(selected_train_cs1["next_availability"]) >= pd.to_datetime(now) - pd.Timedelta(hours = 0))
                                              (pd.to_datetime(selected_train_cs1["next_availability"]) >= pd.to_datetime(now) - pd.Timedelta(hours = 1))
                                              & (pd.to_datetime(selected_train_cs1["next_availability"]) <= pd.to_datetime(now) + pd.Timedelta(hours = 25))
                                            )
                                            |
                                            (
                                              (selected_train_cs1["rake_status"] == "PARKED")
                                            )
                                            ].reset_index(drop = True)

    selected_train_cs2.destination_code = selected_train_cs2.destination_code.fillna(selected_train_cs2.origin_code)

    for i in range(len(selected_train_cs2)):
      if selected_train_cs2.loc[i,"origin_code"] == "NTSJ":
        selected_train_cs2.loc[i,"origin_code"] = "AFAS"
      if selected_train_cs2.loc[i,"destination_code"] == "NTSJ":
        selected_train_cs2.loc[i,"destination_code"] = "AFAS"

    selected_train_cs2 = selected_train_cs2.drop_duplicates(subset = ['rake_name','origin_code','destination_code'],keep = "first").reset_index(drop = True)

    selected_train_cs2.load_plan_teus = selected_train_cs2.load_plan_teus.fillna(0)
    selected_train_cs2.load_plan_teus = selected_train_cs2.load_plan_teus.astype(int)

    train_current_status = selected_train_cs2.reset_index(drop = True)
    train_current_status_copy = train_current_status.copy()

    train_current_status_copy['actual_departure']=pd.to_datetime(train_current_status_copy['actual_departure'])
    train_current_status_copy['FOIS_ETA']=pd.to_datetime(train_current_status_copy['FOIS_ETA'])
    train_current_status_copy['TXR_Due_Date']=pd.to_datetime(train_current_status_copy['TXR_Due_Date']).dt.date
    train_current_status_copy['next_availability']=pd.to_datetime(train_current_status_copy['next_availability'])
    train_current_status_copy['next_available_date']=pd.to_datetime(train_current_status_copy['next_available_date']).dt.date
    train_current_status_copy[['transit_sla','t_tat_sla','t_tat_sla_destination','rational_distance']]=train_current_status_copy[['transit_sla','t_tat_sla','t_tat_sla_destination','rational_distance']].astype(float)
    train_current_status_copy[['unts','TXR_Kms_Remaining','load_plan_teus']]=train_current_status_copy[['unts','TXR_Kms_Remaining','load_plan_teus']].fillna('-99').astype(int)
    #train_current_status_copy=train_current_status_copy.fillna("")
    train_current_status_copy[["rake_name","rake_status","status","origin_code","destination_code","base_depot_code","ss_ds"]]=train_current_status_copy[["rake_name","rake_status","status","origin_code","destination_code","base_depot_code","ss_ds"]].astype(str)
    curr_schema=[{"name": "actual_departure","type": "TIMESTAMP"},{"name": "FOIS_ETA","type": "TIMESTAMP"},{"name": "rake_name","type": "STRING"},{"name": "rake_status","type": "STRING"},{"name": "status","type": "STRING"},{"name": "origin_code","type": "STRING"},{"name": "destination_code","type": "STRING"},{"name": "transit_sla","type": "FLOAT"},{"name": "t_tat_sla","type": "FLOAT"},{"name": "unts","type": "INTEGER"},{"name": "TXR_Due_Date","type": "DATE"},{"name": "TXR_Kms_Remaining","type": "INTEGER"},{"name": "base_depot_code","type": "STRING"},{"name": "load_plan_teus","type": "INTEGER"},{"name": "rational_distance","type": "FLOAT"},{"name": "ss_ds","type": "STRING"},{"name": "t_tat_sla_destination","type": "FLOAT"},{"name": "next_availability","type": "TIMESTAMP"},{"name": "next_available_date","type": "DATE"}]
    train_current_status_copy.to_gbq('apsez-svc-prod-datalake.logistics_cleansed.layer2_bt_rop_train_current_status',project_id='apsez-svc-prod-datalake',table_schema=curr_schema,if_exists='replace')

    print("loaded {0} rows to {1} ".format(train_current_status_copy.shape[0],'logistics_cleansed.layer2_bt_rop_train_current_status'))


    ################################################################################
    # For Pendency Data- Loading BigQuery data for Pendency
    ################################################################################

    query_p = """
    select distinct * from (select pendency_loaded, pendency_empty, og_map.fois as booking_location_code, to_map.fois to_terminal_code, odr_days, cont_size from (select sum(case when pendency_type = 'LOADED' and cont_size = 20 then 1 when pendency_type = 'LOADED' and cont_size = 40 then 2 else 0 end) pendency_loaded,sum(case when pendency_type = 'EMPTY' and cont_size = 20 then 1 when pendency_type = 'EMPTY' and cont_size = 40 then 2 else 0 end) pendency_empty, booking_location_code, to_terminal_code, round(max_age/24) odr_days, cont_size from (SELECT
      *, max(aging_in_hrs) over(partition by booking_location_code, to_terminal_code) max_age,
      CASE
        WHEN cont_size = 20 AND movement_types = 'EMPTY' AND status = 'AWAITING RAILMENT' AND all_empty_flag = 0 AND bu = 'ALL' THEN 'ALL EMPTY'
        WHEN cont_size = 40 AND movement_types = 'EMPTY' AND status = 'AWAITING RAILMENT' AND all_empty_flag = 0 AND bu = 'ALL' THEN 'ALL EMPTY'
        WHEN cont_size = 20 AND movement_types = 'EMPTY' AND status = 'AWAITING RAILMENT' THEN 'EMPTY'
        WHEN cont_size = 40 AND movement_types = 'EMPTY' AND status = 'AWAITING RAILMENT' THEN 'EMPTY'
        WHEN cont_size = 20 AND movement_types = 'EMPTY' AND status = 'INTRANSIT HUB PENDENCY' THEN 'EMPTY (HUB)'
        WHEN cont_size = 40 AND movement_types = 'EMPTY' AND status = 'INTRANSIT HUB PENDENCY' THEN 'EMPTY (HUB)'
        WHEN cont_size = 20 AND movement_types = 'LOADED' AND status = 'AWAITING RAILMENT' THEN 'LOADED'
        WHEN cont_size = 40 AND movement_types = 'LOADED' AND status = 'AWAITING RAILMENT' THEN 'LOADED'
        WHEN cont_size = 20 AND movement_types = 'LOADED' AND status = 'INTRANSIT HUB PENDENCY' THEN 'LOADED (HUB)'
        WHEN cont_size = 40 AND movement_types = 'LOADED' AND status = 'INTRANSIT HUB PENDENCY' THEN 'LOADED (HUB)'
        WHEN cont_size = 20 AND movement_types = 'HOLD' AND status = 'AWAITING RAILMENT' THEN 'ON HOLD'
        WHEN cont_size = 40 AND movement_types = 'LOADED' AND status = 'AWAITING RAILMENT' THEN 'ON HOLD'
        ELSE 'OTHERS'
      END AS pendency_type
    FROM (
      SELECT
        Movement_Type,
        status,
        cont_no,
        cont_size,
        booking_no,
        booking_ref_code,
        customer,
        booking_location,
        current_location,
        to_terminal,
        COALESCE(fois, to_terminal_code) AS to_terminal_code,
        COALESCE(bk.bk_fois, booking_location_code) AS booking_location_code,
        route,
        aging_in_hrs,
        container_status_start_date,
        book_date,
        train_no,
        rake_name,
        transport_by,
        monthyear,
        bu,
        mov_type,
        load_date,
        location,
        movement_types,
        Driven_BU,
        0 AS all_empty_flag
      FROM (
        SELECT
          Movement_Type,
          UPPER(status) AS status,
          cont_no,
          cont_size,
          booking_no,
          booking_ref_code,
          customer,
          booking_location,
          current_location,
          to_terminal,
          to_terminal_code,
          booking_location_code,
          route,
          aging_in_hrs,
          container_status_start_date,
          book_date,
          train_no,
          rake_name,
          transport_by,
          monthyear,
          bu,
          mov_type,
          load_date,
          location,
          CASE
            WHEN Movement_Type = 'Empty' THEN 'EMPTY'
            WHEN Movement_Type = 'Hold' THEN 'HOLD'
            ELSE 'LOADED'
          END AS movement_types,
          CASE
            WHEN UPPER(booking_location_code) IN (
              SELECT terminal_code
              FROM apsez-svc-prod-datalake.logistics_cleansed.layer2_gpwis_terminal_mst
            ) OR UPPER(to_terminal_code) IN (
              SELECT terminal_code
              FROM apsez-svc-prod-datalake.logistics_cleansed.layer2_gpwis_terminal_mst
            ) THEN 'GPWIS'
            ELSE 'ALL+ALSPL'
          END AS Driven_BU,
          0 AS all_empty_flag
        FROM apsez-svc-prod-datalake.logistics_semantic.layer4_rt_rail_pendency_mv a
        WHERE load_date = (
          SELECT MAX(load_date)
          FROM apsez-svc-prod-datalake.logistics_semantic.layer4_rt_rail_pendency_mv
        )
        AND booking_location <> to_terminal
        AND book_date >= '2021-04-01'
        AND customer LIKE '%ADANI LOGISTICS LTD.%'
        AND Movement_Type = 'Empty'
        UNION DISTINCT
        SELECT
          Movement_Type,
          UPPER(status) AS status,
          cont_no,
          cont_size,
          booking_no,
          booking_ref_code,
          customer,
          booking_location,
          current_location,
          to_terminal,
          to_terminal_code,
          CASE WHEN status = 'Intransit Hub Pendency' THEN current_location ELSE booking_location_code END,
          route,
          aging_in_hrs,
          container_status_start_date,
          book_date,
          train_no,
          rake_name,
          transport_by,
          monthyear,
          bu,
          mov_type,
          load_date,
          location,
          CASE
            WHEN Movement_Type = 'Empty' THEN 'EMPTY'
            WHEN Movement_Type = 'Hold' THEN 'HOLD'
            ELSE 'LOADED'
          END AS movement_types,
          CASE
            WHEN UPPER(booking_location_code) IN (
              SELECT terminal_code
              FROM apsez-svc-prod-datalake.logistics_cleansed.layer2_gpwis_terminal_mst
            ) OR UPPER(to_terminal_code) IN (
              SELECT terminal_code
              FROM apsez-svc-prod-datalake.logistics_cleansed.layer2_gpwis_terminal_mst
            ) THEN 'GPWIS'
            ELSE 'ALL+ALSPL'
          END AS Driven_BU,
          1 AS all_empty_flag
        FROM apsez-svc-prod-datalake.logistics_semantic.layer4_rt_rail_pendency_mv b
        WHERE load_date = (
          SELECT MAX(load_date)
          FROM apsez-svc-prod-datalake.logistics_semantic.layer4_rt_rail_pendency_mv
        )
        AND booking_location <> to_terminal
        AND book_date >= '2021-04-01'
        AND customer NOT LIKE '%ADANI LOGISTICS'
      ) x
      LEFT JOIN (
        SELECT *
        FROM (
          SELECT
            terminal_code,
            fois,
            ROW_NUMBER() OVER (PARTITION BY terminal_code) AS rnk
          FROM apsez-svc-prod-datalake.logistics_cleansed.layer2_terminal_master
        ) x
        WHERE rnk = 1
      ) tm ON x.to_terminal_code = tm.terminal_code
      LEFT JOIN (
        SELECT *
        FROM (
          SELECT
            terminal_code as bk_tcode,
            fois as bk_fois,
            ROW_NUMBER() OVER (PARTITION BY terminal_code) AS rnk
          FROM apsez-svc-prod-datalake.logistics_cleansed.layer2_terminal_master
        ) x
        WHERE rnk = 1
      ) bk ON x.booking_location_code = bk.bk_tcode
    )) where status in ('AWAITING RAILMENT', 'INTRANSIT HUB PENDENCY')
    and   pendency_type in ('LOADED', 'EMPTY')
    group by  booking_location_code, to_terminal_code, max_age, cont_size) a left join (select terminal_code, fois from `logistics_cleansed.layer2_terminal_master`
                 qualify row_number() over(partition by terminal_code)  = 1 ) og_map on og_map.terminal_Code = booking_location_code
    left join (select terminal_code, fois from `logistics_cleansed.layer2_terminal_master`
                 qualify row_number() over(partition by terminal_code)  = 1 ) to_map on to_map.terminal_Code = to_terminal_code
    right join (
    select coalesce(og_map.fois, origin_code) as origin_code, coalesce(to_map.fois, destination_code) destination_code, route, rational_distance, critical_non_critical, ss_ds, r_tat_sla, t_tat_sla  from `logistics_cleansed.layer2_route_sla_mst` rsm
    left join (select terminal_code, fois from `logistics_cleansed.layer2_terminal_master`
                 qualify row_number() over(partition by terminal_code)  = 1 ) og_map
    on og_map.terminal_code = rsm.origin_code
    left join (select terminal_code, fois from `logistics_cleansed.layer2_terminal_master`
                 qualify row_number() over(partition by terminal_code)  = 1 ) to_map
    on to_map.terminal_code = rsm.destination_code) sla on sla.origin_code = og_map.fois and sla.destination_code = to_map.fois
    )x where (pendency_loaded is not null or pendency_empty is not null)"""

    # query_p = "select * from logistics_cleansed.layer2_train_pendency_data"
    query_job_p=client.query(query_p).result()
    pendency = query_job_p.to_dataframe()
    pendency.rename(columns = {"pendency_loaded":"pendency","pendency_empty":"empty_pendency"}, inplace = True)
    pendency.head()

    pendency = pendency.merge(selected_train_cs2[["origin_code","destination_code","load_plan_teus"]], left_on = ["booking_location_code","to_terminal_code"], right_on = ["origin_code","destination_code"], how = "left")
    pendency.pendency = pendency.pendency.astype(int)
    pendency.load_plan_teus = pendency.load_plan_teus.fillna(0)
    pendency.load_plan_teus = pendency.load_plan_teus.astype(int)
    pendency.pendency = pendency.pendency - pendency.load_plan_teus
    pendency.empty_pendency = pendency.empty_pendency.fillna(0)
    pendency.empty_pendency = pendency.empty_pendency.astype(int)
    pendency = pendency[["pendency","empty_pendency","booking_location_code","to_terminal_code","odr_days","cont_size"]].reset_index(drop = True)

    selected_pendency = pendency.copy()

    for i in range(len(selected_pendency)):
      if selected_pendency.loc[i,"booking_location_code"] == "NTSJ":
        selected_pendency.loc[i,"booking_location_code"] = "AFAS"
      if selected_pendency.loc[i,"to_terminal_code"] == "NTSJ":
        selected_pendency.loc[i,"to_terminal_code"] = "AFAS"

    selected_pendency = selected_pendency.drop_duplicates(subset = ['booking_location_code','to_terminal_code'],keep = "first").reset_index(drop = True)

    pendency_data = selected_pendency.reset_index(drop = True)
    pendency_data_copy = pendency_data.copy()

    pendency_data_copy.to_gbq('apsez-svc-prod-datalake.logistics_cleansed.layer2_bt_rop_pendency_data',project_id='apsez-svc-prod-datalake',if_exists='replace')

    print("loaded {0} rows to {1} ".format(pendency_data_copy.shape[0],'logistics_cleansed.layer2_bt_rop_pendency_data'))

    ################################################################################
    # For Via Route Data- Loading BigQuery data for Via Route
    ################################################################################

    query_p = """select distinct * from logistics_master.layer2_via_route_master"""

    query_job_p=client.query(query_p).result()
    via_rt = query_job_p.to_dataframe()
    via_rt.rename(columns = {"source":"origin_code","destination":"destination_code"}, inplace = True)
    via_rt = via_rt.groupby(['origin_code','destination_code','route_type']).via.agg([('via','/'.join)]).reset_index()
    via_rt["route"] = via_rt["origin_code"] + "-" + via_rt["destination_code"]
    via_rt["via_route"] = via_rt["origin_code"] + "-" + via_rt["via"] + "-" + via_rt["destination_code"]


    ################################################################################
	# Data Prep for Optimiser
	################################################################################
	################################################################################
	# Data Prep for Optimiser
	################################################################################
	################################################################################
	# Data Prep for Optimiser
	################################################################################

	import pandas as pd
	import numpy as np
	import warnings
	warnings.simplefilter("ignore")
	pd.set_option('display.max_rows', 500)
	pd.set_option('display.max_columns', 1000)
	from datetime import datetime, timedelta

	today_filename = datetime.today().strftime("%d%m%Y")

	# train_current_status_file_name = "/content/drive/MyDrive/Filesfrom27thOct/"+today_filename+"_train_current_status.csv"
	# distance_data_file_name = "/content/drive/MyDrive/Filesfrom27thOct/"+today_filename+"_distance_data.csv"
	# pendency_data_file_name = "/content/drive/MyDrive/Filesfrom27thOct/"+today_filename+"_pendency_data.csv"
	# # train_current_status_file_name = "/content/drive/MyDrive/Filesfrom27thOct/"+"19122023"+"_train_current_status.csv"
	# # distance_data_file_name = "/content/drive/MyDrive/Filesfrom27thOct/"+"19122023"+"_distance_data.csv"
	# # pendency_data_file_name = "/content/drive/MyDrive/Filesfrom27thOct/"+"19122023"+"_pendency_data.csv"
	# train_current_status1 = pd.read_csv(train_current_status_file_name)
	# distance_data = pd.read_csv(distance_data_file_name)
	# pendency_data = pd.read_csv(pendency_data_file_name)

	today = datetime.today().strftime("%Y-%m-%d")
	train_current_status1 = train_current_status_copy.reset_index(drop = True)
	pendency_data = pendency_data_copy.reset_index(drop = True)
	distance_data = distance_data_copy.reset_index(drop = True)

	# dates in date format
	train_current_status1["TXR_Due_Date"] = pd.to_datetime(train_current_status1["TXR_Due_Date"])
	train_current_status1["next_availability"] = pd.to_datetime(train_current_status1["next_availability"])
	train_current_status1["next_available_date"] = pd.to_datetime(train_current_status1["next_available_date"])
	train_current_status1["flag"] = 1

	# drop the rows from pendency data if destination or origin is na.
	pendency_data["flag"] = 1
	for i in range(len(pendency_data)):
	  if ((pd.isna(pendency_data.loc[i,"booking_location_code"]))) or (pd.isna(pendency_data.loc[i,"booking_location_code"])) or (pd.isna(pendency_data.loc[i,"pendency"])):
		pendency_data.loc[i,"flag"] = 0
	  if ((pendency_data.loc[i,"pendency"] <= 0) & (pendency_data.loc[i,"empty_pendency"] <= 0)):
		pendency_data.loc[i,"flag"] = 0
	pendency_data = pendency_data[pendency_data.flag == 1].reset_index(drop = True)

	# drop the rows from distance data if destination or origin is na.
	distance_data.rename(columns = {"active_rt":"Active_Route"}, inplace = True)
	distance_data["flag"] = 1
	# take only the active routes
	distance_data = distance_data[distance_data.Active_Route == 1].reset_index(drop = True)
	for i in range(len(distance_data)):
	  if ((pd.isna(distance_data.loc[i,"origin_code"]))) or (pd.isna(distance_data.loc[i,"destination_code"])) or (pd.isna(distance_data.loc[i,"rational_distance"])):
		distance_data.loc[i,"flag"] = 0
	  if ((distance_data.loc[i,"origin_code"] == "AFAS") & (distance_data.loc[i,"destination_code"] == "PPSP")):
		distance_data.loc[i,"flag"] = 0
	  if ((distance_data.loc[i,"origin_code"] == "PPSP") & (distance_data.loc[i,"destination_code"] == "AFAS")):
		distance_data.loc[i,"flag"] = 0
	  if ((distance_data.loc[i,"origin_code"] == "MDCC") & (distance_data.loc[i,"destination_code"] == "PPSP")):
		distance_data.loc[i,"flag"] = 0
	distance_data = distance_data[distance_data.flag == 1].reset_index(drop = True)

	loc = list(set(list(distance_data.origin_code) + list(distance_data.destination_code)))
	for i in range(len(train_current_status1)):
	  if train_current_status1.loc[i,"origin_code"] not in loc:
		train_current_status1.loc[i,"flag"] = 0
	  if train_current_status1.loc[i,"destination_code"] not in loc:
		train_current_status1.loc[i,"flag"] = 0
	  if (train_current_status1.loc[i,"TXR_Due_Date"] is pd.NaT):
		train_current_status1.loc[i,"flag"] = 0
	  if ((pd.isna(train_current_status1.loc[i,"TXR_Kms_Remaining"]))|
	   ((train_current_status1.loc[i,"TXR_Kms_Remaining"]) == "None")|
		((train_current_status1.loc[i,"TXR_Kms_Remaining"]) == "NA")):
		train_current_status1.loc[i,"TXR_Kms_Remaining"] = -1
		train_current_status1.loc[i,"flag"] = 0
	  if (train_current_status1.loc[i,"TXR_Kms_Remaining"] == "END TO END"):
		train_current_status1.loc[i,"TXR_Kms_Remaining"] = -1
	  if ((train_current_status1.loc[i,"destination_code"] == train_current_status1.loc[i,"base_depot_code"]) &
	   ((train_current_status1.loc[i,"TXR_Due_Date"] <= pd.to_datetime(today)))):
	   train_current_status1.loc[i,"TXR_Due_Date"] = pd.to_datetime(today)
	  if ((train_current_status1.loc[i,"destination_code"] == train_current_status1.loc[i,"base_depot_code"]) & (int(train_current_status1.loc[i,"TXR_Kms_Remaining"]) <= 0)):
	   train_current_status1.loc[i,"TXR_Kms_Remaining"] = -1
	  if ((train_current_status1.loc[i,"destination_code"] == train_current_status1.loc[i,"origin_code"])):
	   train_current_status1.loc[i,"rational_distance"] = 0

	train_current_status = train_current_status1[train_current_status1.flag == 1].reset_index(drop = True)
	train_current_status["TXR_Kms_Remaining"] = train_current_status.TXR_Kms_Remaining.astype('float64').round(0)
	train_current_status["rational_distance"] = train_current_status["rational_distance"].astype('float64').round(0)
	train_current_status["TXR_KmsRem_CurrDest"] = train_current_status["TXR_Kms_Remaining"] - train_current_status1["rational_distance"]
	# train_current_status["Manual_Plan"] = 0


	# # FRONTEND DEVELOPMENT :

	# if len(custom_plan_df) > 0:
	#   schedule2_rakes = custom_plan_df.rake_name.to_list()
	#   for i in range(len(train_current_status)):
	#     if train_current_status.loc[i,"rake_name"].isin(schedule2_rakes):
	#       train_current_status.loc[i,"Manual_Plan"] = 1
	train_current_status_copy1 = train_current_status.copy() #copy of all flag 1 trains
	# train_current_status = train_current_status[train_current_status.Manual_Plan == 0].reset_index(drop = True)
	# schedule2 =


	# ##############
	# ##############
	# ##############
	# ##############
	# ##############
	# ##############
	# ##############
	# ##############
	# ##############
	# ##############



	from ortools.sat.python import cp_model
	import pandas as pd
	import pandas as pd
	import numpy as np
	import warnings
	warnings.simplefilter("ignore")
	pd.set_option('display.max_rows', 500)
	pd.set_option('display.max_columns', 1000)
	from datetime import datetime, timedelta

	################################################################################
	# OR Tools
	################################################################################

	df_dist_time = distance_data.reset_index(drop = True)
	df_pendency = pendency_data.reset_index(drop = True)
	df_train_curr_plan = train_current_status.reset_index(drop = True)

	# class OptTrainRoute:
	#   def _init_(self,df_dist_time,df_pendency,df_train_curr_plan):
	#     self.Weightage_Load_Pend = 5
	#     self.CALCULATE_ALL_PAIRS_SHORTEST = False
	#     self.CALCULATE_ALL_BASE_LOC_PAIRS_SHORTEST = True
	#     self.INF = 1000000000
	#     self.ndist = INF # default distance if a route doesn't exists between two points
	#     self.ntime = INF # default time if a route doesn't exists between two points
	#     self.nstack = 2 # Default is considered SS
	#     self.npendency = 0 # Default pendency if path doesn't exist between locations
	#     self.M = 1 # Increase the pendency of closed_routes so as to incentivize the solver to pick trains on such routes.
	#     self.df_dist_time = df_dist_time
	#     self.df_pendency = df_pendency
	#     self.df_train_curr_plan = df_train_curr_plan

	Weightage_Load_Pend = 5
	CALCULATE_ALL_PAIRS_SHORTEST = False
	CALCULATE_ALL_BASE_LOC_PAIRS_SHORTEST = True
	INF = 1000000000
	ndist = INF # default distance if a route doesn't exists between two points
	ntime = INF # default time if a route doesn't exists between two points
	nstack = 2 # Default is considered SS
	npendency = 0 # Default pendency if path doesn't exist between locations
	cont_dict = {20: 0, 40: 1}
	overall_pendency_data = pendency_data.groupby(["booking_location_code","to_terminal_code"])[["pendency","empty_pendency"]].agg('sum').reset_index()
	overall_pendency_data["route"] =  overall_pendency_data["booking_location_code"] + "-" + overall_pendency_data["to_terminal_code"]

	def data_prep(df_dist_time, df_pendency, df_train_curr_plan):
		### Distance-Time data
		# In stack, SS = 90 and DS = 180 .
		# Dist is the rounded off Distance
		# Time_min is in minute
		# Null values in Time_min is replaced by (distance between this route / average speed of train)
		df_dist_time.rename(columns={'origin_code':'Origin','destination_code':'Destination','rational_distance':'Distance'}, inplace=True)

		df_dist_time['Stack'] = df_dist_time['ss_ds'].apply(lambda x: 90 if x == 'SS' else (180 if x == 'DS' else None))

		df_dist_time['Time_min'] = round(60 * df_dist_time['r_tat_sla'])
		df_dist_time['Dist'] = round(df_dist_time['Distance'])
		valid_rows = df_dist_time['Time_min'].notnull()  # boolean mask for non-null values in 'time'
		average_speed = df_dist_time.loc[valid_rows, 'Dist'].sum() / df_dist_time.loc[valid_rows, 'Time_min'].sum()
		df_dist_time['Time_min'].fillna(df_dist_time['Dist'] / average_speed,inplace=True)
		'''# Active_Route
		df_dist_time['Active_Route'] = 1
		# Closed Route
		df_dist_time['closed_rt'] = 0
		df_dist_time.loc[(df_dist_time['Origin'] == 'PXXL') & (df_dist_time['Destination'] == 'ZZBM'), 'closed_rt'] = 1
		df_dist_time.loc[(df_dist_time['Origin'] == 'MIIC') & (df_dist_time['Destination'] == 'QQPQ'), 'closed_rt'] = 1
		# df_dist_time.loc[(df_dist_time['Origin'] == 'MIIC') & (df_dist_time['Destination'] == 'SSKA'), 'closed_rt'] = 1
		df_dist_time.loc[(df_dist_time['Origin'] == 'ALPK') & (df_dist_time['Destination'] == 'UIV'), 'closed_rt'] = 1
		'''
		dist_time_df = df_dist_time.copy()

		### pendency data
		df_pendency.rename(columns={'pendency':'Pendency','to_terminal_code':'Destination','booking_location_code':'Source'}, inplace=True)
		'''
		df_pendency['empty_pendency'] = np.random.uniform(1, 100, size = df_pendency.shape[0])
		df_pendency['empty_pendency'] = round(df_pendency['empty_pendency'])
		df_pendency['cont_size'] = 20
		new_data = {'Pendency': [1500, 15], 'Source': ['KDPO', 'QQPQ'], 'Destination': ['APPS', 'PXXL'], 'odr_days': [6,8], 'empty_pendency':[35,75],'cont_size':[40,40]}
		# Creating a new DataFrame with the dummy data for 40 ft containers ##pen41
		new_rows_df = pd.DataFrame(new_data)
		# Appending the new rows to the existing DataFrame
		df_pendency = pd.concat([df_pendency, new_rows_df], ignore_index = True)
		'''
		df_pendency['Pendency'] = df_pendency['Pendency'].apply(lambda x: max(0, x))
		df_pendency['empty_pendency'] = df_pendency['empty_pendency'].apply(lambda x: max(0, x))
		pend_df = df_pendency.copy()


		### Train Current Status data
		# Rational distance value are replaced by 0 because these trains were at the Terminal. They didn't covered any distance.
		# Actual remaining km = Remaining_kms - Distance already covered(rational_distance)
		df_train_curr_plan.rename(columns={'FOIS_ETA':'ETA','rake_name':'Train','origin_code':'Origin','destination_code':'Destination','TXR_Kms_Remaining':'Remaining_kms'},inplace=True)
		df_train_curr_plan['rational_distance'].fillna(0,inplace=True)
		df_train_curr_plan['Actual_remaining_kms'] = round(df_train_curr_plan['Remaining_kms'] - df_train_curr_plan['rational_distance'])
		df_train_curr_plan["remaining_min"] = pd.to_datetime(df_train_curr_plan["TXR_Due_Date"]) - pd.to_datetime(df_train_curr_plan["ETA"])
		df_train_curr_plan["remaining_min"] = round((60*df_train_curr_plan["remaining_min"])//pd.Timedelta(hours=1))
		# Replace negative values in 'remaining_min' with 0
		# df_train_curr_plan['remaining_min'] = df_train_curr_plan['remaining_min'].apply(lambda x: max(0, x))
		# Replace negative values in 'Actual_remaining_kms' with 0
		# df_train_curr_plan['Actual_remaining_kms'] = df_train_curr_plan['Actual_remaining_kms'].apply(lambda x: max(0, x))
		# df_train_curr_plan.loc[(df_train_curr_plan['Train'] == '11XX'), 'remaining_min'] = 1000
		train_curr_plan_df = df_train_curr_plan.copy()

		return dist_time_df, pend_df, train_curr_plan_df


	def get_loc(df, df1, df2):
		src = sorted(list(df['Origin'].tolist())) # Dist_Time
		dest = sorted(list(df['Destination'].tolist())) # Dist_Time
		src1 = sorted(list(df1['Source'].tolist())) # Pendency
		dest1 = sorted(list(df1['Destination'].tolist())) # Pendency
		dest2 = sorted(list(df2['Destination'].tolist())) # Train_Curr_Status
		base_depots = sorted(list(df2['base_depot_code'].tolist())) # Train_Curr_Status
		locs = sorted(list(set((src) + (dest) + (src1) + (dest1) + (dest2) + (base_depots)))) # get unique locations
		loc_dict = {}
		for idx, loc in enumerate(locs):
			loc_dict[loc] = idx
		return locs, loc_dict

	def get_base_depot(df, trains, train_dict):
		base_depot_dict = {}
		for idx in range(df.shape[0]):
			train, base_depot = df.at[idx, 'Train'], df.at[idx, 'base_depot_code']
			base_depot_dict[train] = base_depot
		base_depot_locs = list(df['base_depot_code'].unique().tolist())
		return base_depot_locs, base_depot_dict

	def get_dist_time_stack(df, num_loc, loc_dict, locs, base_loc, Trains, train_dict, start_loc):
		dist = [[ndist] * num_loc for _ in range(num_loc)]
		time = [[ntime] * num_loc for _ in range(num_loc)]
		closed_rt = {}
		for i in range(num_loc):
			dist[i][i], time[i][i] = 0, 0
		stack = [[nstack] * num_loc for _ in range(num_loc)]
		for idx in range(df.shape[0]):
			src, dest, d, t, s, actve, close_rt = df.at[idx, 'Origin'], df.at[idx, 'Destination'], df.at[idx, 'Dist'], df.at[idx, 'Time_min'], df.at[idx, 'ss_ds'], df.at[idx, 'Active_Route'], df.at[idx, 'closed_rt']
			val = 2
			if s == 'DS':
				val = 4
			dist[loc_dict[src]][loc_dict[dest]] = int(d)
			# dist[loc_dict[dest]][loc_dict[src]] = d
			time[loc_dict[src]][loc_dict[dest]] = int(t)
			# time[loc_dict[dest]][loc_dict[src]] = t
			stack[loc_dict[src]][loc_dict[dest]] = val
			stack[loc_dict[dest]][loc_dict[src]] = val
			if int(close_rt) == 1:
				closed_rt[(src, dest)] = 1

		closed_rt_train = {}
		for train in Trains:
			loc1 = start_loc[train]
			k = train_dict[train]
			closed_rt_train[k] = 0
			for loc2 in locs:
				if (loc1, loc2) in closed_rt:
					closed_rt_train[k] = 1
					break

		n = len(dist)
		if CALCULATE_ALL_BASE_LOC_PAIRS_SHORTEST:
			# Calculate all-pairs shortest path
			# but only consider base_locations
			# as i -> base or j -> base in (i -> j -> base) can be active or inactive
			for k in range(n):
				for loc1 in locs:
					for bloc2 in base_loc:
						i, j = loc_dict[loc1], loc_dict[bloc2]
						if dist[i][k] < INF and dist[k][j] < INF:
							dist[i][j] = min(dist[i][j], dist[i][k] + dist[k][j])
						if time[i][k] < INF and time[k][j] < INF:
							time[i][j] = min(time[i][j], time[i][k] + time[k][j])
		elif CALCULATE_ALL_PAIRS_SHORTEST:
			# Calculate all-pairs shortest path
			for k in range(n):
				for i in range(n):
					for j in range(n):
						if dist[i][k] < INF and dist[k][j] < INF:
							dist[i][j] = min(dist[i][j], dist[i][k] + dist[k][j])
						if time[i][k] < INF and time[k][j] < INF:
							time[i][j] = min(time[i][j], time[i][k] + time[k][j])
		return dist, time, stack, closed_rt, closed_rt_train

	def get_trains_rem_dist_time(df):
		Trains = df['Train'].unique().tolist()
		train_dict = {}
		for idx, train in enumerate(Trains):
			train_dict[train] = idx
		wagons = {}
		Rem_Dist = {}
		Rem_Time = {}
		start_loc = {}
		Eta = {}
		for idx in range(df.shape[0]):
			train, rem_km, rem_min, starting_loc, wg, eta = df.at[idx, 'Train'], df.at[idx, 'Actual_remaining_kms'], df.at[idx, 'remaining_min'], df.at[idx, 'Destination'], df.at[idx, 'unts'], df.at[idx, 'ETA']
			Rem_Dist[train_dict[train]] = int(rem_km)
			Rem_Time[train_dict[train]] = int(rem_min)
			wagons[train_dict[train]] = int(wg)
			start_loc[train] = starting_loc
			Eta[train_dict[train]] = str(eta)
		return Trains, train_dict, Rem_Dist, Rem_Time, start_loc, wagons, Eta

	def get_pendency(df, num_loc, loc_dict):
		pend = [[[npendency, npendency] for _ in range(num_loc)] for _ in range(num_loc)]
		empty_pend = [[[npendency, npendency] for _ in range(num_loc)] for _ in range(num_loc)]
		for idx in range(df.shape[0]):
			src, dest, pendency, e_pend, cont_sz = df.at[idx, 'Source'], df.at[idx, 'Destination'], df.at[idx, 'Pendency'], df.at[idx, 'empty_pendency'], df.at[idx, 'cont_size']
			x, y= loc_dict[src], loc_dict[dest]
			pend[x][y][cont_dict[cont_sz]] = int(pendency)
			# pend[y][x] = pendency
			empty_pend[x][y][cont_dict[cont_sz]] = int(e_pend)
		pend_sums = []
		for i in range(len(pend)):
			nsum = 0
			for j in range(len(pend[i])):
				nsum += pend[i][j][0] + pend[i][j][1]
			pend_sums.append(nsum)
		return pend, pend_sums, empty_pend

	def get_pend_updated(pend_df, df_out):
	  if len(df_out) > 0:
		df_pend_cleared_20 = df_out.groupby(['Source', 'Destination'], as_index=False)['Loaded_Pend_Cleared_20'].sum()
		df_empty_cleared_20 = df_out.groupby(['Source', 'Destination'], as_index=False)['Empty_Pend_Cleared_20'].sum()
		df_pend_cleared_40 = df_out.groupby(['Source', 'Destination'], as_index=False)['Loaded_Pend_Cleared_40'].sum()
		df_empty_cleared_40 = df_out.groupby(['Source', 'Destination'], as_index=False)['Empty_Pend_Cleared_40'].sum()
		df_merged = pd.merge(df_pend_cleared_20, pend_df, on=['Source', 'Destination'], how='right')
		df_merged_1 = pd.merge(df_empty_cleared_20, df_merged, on=['Source', 'Destination'], how='right')
		df_merged_40 = pd.merge(df_pend_cleared_40, df_merged_1, on=['Source', 'Destination'], how='right')
		df_merged_1_40 = pd.merge(df_empty_cleared_40, df_merged_40, on=['Source', 'Destination'], how='right')
		cols_type_int = ['Loaded_Pend_Cleared_40','Loaded_Pend_Cleared_20','Empty_Pend_Cleared_40','Empty_Pend_Cleared_20']
		df_merged_1_40[cols_type_int] = df_merged_1_40[cols_type_int].replace('', 0)
		df_merged_1_40['pen_units_remaining'] =np.where(df_merged_1_40['cont_size']==40,df_merged_1_40['Pendency'] - df_merged_1_40['Loaded_Pend_Cleared_40'] ,df_merged_1_40['Pendency'] - df_merged_1_40['Loaded_Pend_Cleared_20'])
		df_merged_1_40['pen_units_remaining'].fillna(df_merged_1_40['Pendency'],inplace=True)
		df_merged_1_40['emp_units_remaining'] =np.where(df_merged_1_40['cont_size']==40,df_merged_1_40['empty_pendency'] - df_merged_1_40['Empty_Pend_Cleared_40'] ,df_merged_1_40['empty_pendency'] - df_merged_1_40['Empty_Pend_Cleared_20'])
		df_merged_1_40['emp_units_remaining'].fillna(df_merged_1_40['empty_pendency'],inplace=True)
		pend_df_updated = df_merged_1_40[['Source','Destination','pen_units_remaining','emp_units_remaining','odr_days','cont_size']].copy()
		pend_df_updated.rename(columns={'pen_units_remaining':'Pendency','emp_units_remaining':'empty_pendency' }, inplace=True)
		pend_df_updated.head()
	  else:
		pend_df_updated = pend_df
	  return pend_df_updated

	def optimize(num_trains, num_stations, num_base, num_loc, locs, Trains, train_dict, dist,
						  time, Rem_Dist, Rem_Time, loc_dict, base_loc, base_depot_dict, pend, empty_pend, stack, start_loc,
						  pend_sums, wagons, Eta, closed_rt, closed_rt_train, dist_time_df, pend_df, train_curr_plan_df):

		# Create the CP-SAT model
		# Uses Google CP-SAT Solver
		model = cp_model.CpModel()
		# x_{kij} = 1 if Train k is assigned location pair (i,j) else 0.
		x = {}
		# y_{kij} -> Loaded_Pendency cleared by train k from i -> j
		# t_{kij} -> Empty_Pendency cleared by train k from i -> j
		# Upd: 03/01/24 -> Take care of 20ft and 40ft container.
		# {y/t}_{kij20} and {y/t}_{kij40}
		# {i, j} can be base location as well
		# Note: Different num_wagons for trains can be dealt here.
		# stack[i][j] will be num_wagons[k] * {SS, DS} for k^th train.
		y = {}
		t = {}
		train_start_locs = []
		for train in Trains:
			k = train_dict[train]
			expr = []
			loc1 = start_loc[train]
			train_start_locs.append(loc1)
			for loc2 in locs:
				if loc1 == loc2:
					continue
				i, j = loc_dict[loc1], loc_dict[loc2]
				x[(k, i, j)] = model.NewBoolVar(f"x_{k}{i}{j}")
				capacity = wagons[train_dict[train]] * stack[i][j]
				total_loaded_pend = pend[i][j][cont_dict[20]] + pend[i][j][cont_dict[40]] # 20 ft + 40 ft
				lb_20, lb_40 = 0, 0
				# If total_loaded_pendency[i][j] > 0
				# then each scheduled train on [i, j] must carry > 0 units.
				# Note: This behaviour is not applicable if (i->j) is a closed_route.
				if pend[i][j][cont_dict[20]] and (start_loc[train], loc2) not in closed_rt:
					lb_20 = 1
				if pend[i][j][cont_dict[40]] and (start_loc[train], loc2) not in closed_rt:
					lb_40 = 1
				y[(k, i, j, 20)] = model.NewIntVar(lb_20, min(pend[i][j][cont_dict[20]], capacity), f"y_{k}{i}{j}{20}") # 20 ft
				y[(k, i, j, 40)] = model.NewIntVar(lb_40, min(pend[i][j][cont_dict[40]], capacity), f"y_{k}{i}{j}{40}") # 40 ft

				for cont_size in [20, 40]:
					yx = model.NewIntVar(0, min(pend[i][j][cont_dict[cont_size]], capacity), f"yx{k}{i}{j}{cont_size}")
					model.AddMultiplicationEquality(yx, x[(k, i, j)], y[(k, i, j, cont_size)])
					b = model.NewBoolVar('b')
					model.AddModuloEquality(b, yx, 2)
					model.Add(b == 0)

				lb = 0
				t[(k, i, j, 20)] = model.NewIntVar(lb, min(empty_pend[i][j][cont_dict[20]], capacity), f"t_{k}{i}{j}{20}")
				t[(k, i, j, 40)] = model.NewIntVar(lb, min(empty_pend[i][j][cont_dict[40]], capacity), f"t_{k}{i}{j}{40}")
				# Make empties_cleared for 20 ft and 40 ft container 'even' if path (i->j) is chosen
				for cont_size in [20, 40]:
					tx = model.NewIntVar(0, min(empty_pend[i][j][cont_dict[cont_size]], capacity), f"tx{k}{i}{j}{cont_size}")
					model.AddMultiplicationEquality(tx, x[(k, i, j)], t[(k, i, j, cont_size)])
					b1 = model.NewBoolVar('b1')
					model.AddModuloEquality(b1, tx, 2)
					model.Add(b1 == 0)
				expr.append(x[(k, i, j)])
			# A train can be assigned to at-most 1 route.
			model.Add(cp_model.LinearExpr.Sum(expr) <= 1)

		train_start_locs = list(set(train_start_locs))

		# If no route (i->j) exists, it won't be considered.
		for train in Trains:
			k = train_dict[train]
			i = loc_dict[start_loc[train]]
			for loc in locs:
				j = loc_dict[loc]
				if i == j: # or loc == base_depot_dict[train]
					continue
				if dist[i][j] == INF or time[i][j] == INF:
					model.Add(x[(k, i, j)] == 0)

		'''
		# Weak Maintenance Constraint
		# for closed routes (i -> j)
		for (loc, val) in closed_rt.items():
			loc1, loc2 = loc
			i, j = loc_dict[loc1], loc_dict[loc2]
			for train in Trains:
				if loc1 != start_loc[train]:
					continue
				k = train_dict[train]
				model.Add(x[(k, i, j)] * dist[i][j] <= Rem_Dist[k])
				model.Add(x[(k, i, j)] * time[i][j] <= Rem_Time[k])
		'''

		# Deal with closed_rt
		# For a particular start_loc i, there exists a single j
		# such that i->j is a closed route, then the train
		# must consider that j
		for (loc, val) in closed_rt.items():
			loc1, loc2 = loc
			i, j = loc_dict[loc1], loc_dict[loc2]
			flag = 0
			for train in Trains:
				if loc1 != start_loc[train]:
					continue
				k = train_dict[train]
				model.Add(x[(k, i, j)] == 1)

		# Maintenance constraint(strong, makes sure that after the next trip train can
		# still return for maintenance without violating constraint):
		# This doesn't consider train going directly to base location.
		# Distance and Time
		for train in Trains:
			k = train_dict[train]
			loc1 = start_loc[train]
			for loc2 in locs:
				i, j, base_depot_idx = loc_dict[loc1], loc_dict[loc2], loc_dict[base_depot_dict[train]]
				if loc1 == loc2 or loc2 == base_depot_dict[train]:
				  continue
				# This takes care of i->j exists but j->base doesn't
				if dist[j][base_depot_idx] == INF or time[j][base_depot_idx] == INF:
					model.Add(x[(k, i, j)] == 0)
					continue
				# if (i->j) closed_route then we are
				# not checking strong_maintenance constraint
				if (start_loc[train], loc2) in closed_rt:
					continue
				model.Add((dist[j][base_depot_idx] * x[(k, i, j)]) <= Rem_Dist[k] - (dist[i][j] * x[(k, i, j)])).OnlyEnforceIf(x[(k, i, j)])
				model.Add((time[j][base_depot_idx] * x[(k, i, j)]) <= Rem_Time[k] - (time[i][j] * x[(k, i, j)])).OnlyEnforceIf(x[(k, i, j)])


		# Deal with the {dist, time} constraint if train directly goes to base location.
		# Not being considered presently.
		# Train can always directly go to base location even if it doesn't have
		# enough distance/time or even if there isn't any route.
		'''
		for train in Trains:
			k, i, j = train_dict[train], loc_dict[start_loc[train]], loc_dict[base_depot_dict[train]]
			if i == j:
				continue
			model.AddLinearConstraint(dist[i][j] * x[(k, i, j)], 0, Rem_Dist[k])
			model.AddLinearConstraint(time[i][j] * x[(k, i, j)], 0, Rem_Time[k])
		'''
		# For each pair of location (i, j), the sum of {loaded, empty}_pendency (for 20 ft and 40 ft containers)
		# cleared across i->j by all chosen trains
		# should be less than equal to the amount of {loaded, empty}_pendency from i->j (for 20 ft and 40 ft containers separately).
		load_expr = []
		empty_expr = []
		for loc1 in train_start_locs:
			for loc2 in locs:
				if loc1 == loc2:
					continue
				expr_20 = []
				expr_40 = []
				expr1_20 = []
				expr1_40 = []
				for train in Trains:
					if loc1 != start_loc[train]:
						continue
					k, i, j = train_dict[train], loc_dict[loc1], loc_dict[loc2]
					capacity = wagons[train_dict[train]] * stack[i][j]

					yx20 = model.NewIntVar(0, min(pend[i][j][cont_dict[20]], capacity ), f"yx{k}{i}{j}{20}")
					model.AddMultiplicationEquality(yx20, x[(k, i, j)], y[(k, i, j, 20)])
					load_expr.append(yx20)
					expr_20.append(yx20)

					yx40 = model.NewIntVar(0, min(pend[i][j][cont_dict[40]], capacity ), f"yx{k}{i}{j}{40}")
					model.AddMultiplicationEquality(yx40, x[(k, i, j)], y[(k, i, j, 40)])
					load_expr.append(yx40)
					expr_40.append(yx40)

					tx20 = model.NewIntVar(0, min(empty_pend[i][j][cont_dict[20]], capacity), f"tx{k}{i}{j}{20}")
					model.AddMultiplicationEquality(tx20, x[(k, i, j)], t[(k, i, j, 20)])
					empty_expr.append(tx20)
					expr1_20.append(tx20)

					tx40 = model.NewIntVar(0, min(empty_pend[i][j][cont_dict[40]], capacity), f"tx{k}{i}{j}{40}")
					model.AddMultiplicationEquality(tx40, x[(k, i, j)], t[(k, i, j, 40)])
					empty_expr.append(tx40)
					expr1_40.append(tx40)

				model.Add(cp_model.LinearExpr.Sum(expr_20) <= (pend[i][j][cont_dict[20]]))
				model.Add(cp_model.LinearExpr.Sum(expr_40) <= (pend[i][j][cont_dict[40]]))
				model.Add(cp_model.LinearExpr.Sum(expr1_20) <= (empty_pend[i][j][cont_dict[20]]))
				model.Add(cp_model.LinearExpr.Sum(expr1_40) <= (empty_pend[i][j][cont_dict[40]]))

		# Deal with Empty_Pendency
		# if (loaded_pendency < 50 % of capacity
		# then 100 % capacity to be utilized through empty_pendency)
		# or (loaded_pendency >= 50 %)
		# Note: For double stack, this 50 % becomes 25 %
		bt = {}
		for train in Trains:
			k, i = train_dict[train], loc_dict[start_loc[train]]
			for loc2 in locs:
				j = loc_dict[loc2]
				if i == j or start_loc[train] == loc2:
					continue
				bt[(k, i, j)] = model.NewBoolVar(f'bt{k}{i}{j}')
				capacity = wagons[train_dict[train]] * stack[i][j]

				yx20 = model.NewIntVar(0, min(pend[i][j][cont_dict[20]], capacity ), f"yx{k}{i}{j}{20}")
				model.AddMultiplicationEquality(yx20, x[(k, i, j)], y[(k, i, j, 20)])
				yx40 = model.NewIntVar(0, min(pend[i][j][cont_dict[40]], capacity ), f"yx{k}{i}{j}{40}")
				model.AddMultiplicationEquality(yx40, x[(k, i, j)], y[(k, i, j, 40)])
				yx = model.NewIntVar(0, min(pend[i][j][cont_dict[20]] + pend[i][j][cont_dict[40]], capacity ), f"yx{k}{i}{j}")
				model.Add(yx == yx20 + yx40)

				# bt = 1 => loaded_pendency >= 50 % of capacity
				# Note: For double stack, this 50 % becomes 25 %
				# Update 04/01/24: 50 % becomes 100% and 25% becomes 50%
				factor = 1
				if stack[i][j] == 4:
					factor = 2
				model.Add(factor * yx >= capacity).OnlyEnforceIf(bt[(k, i, j)])
				model.Add(factor * yx <= (capacity - 1)).OnlyEnforceIf(bt[(k, i, j)].Not())

				tx20 = model.NewIntVar(0, min(empty_pend[i][j][cont_dict[20]], capacity), f"tx{k}{i}{j}{20}")
				model.AddMultiplicationEquality(tx20, x[(k, i, j)], t[(k, i, j, 20)])
				tx40 = model.NewIntVar(0, min(empty_pend[i][j][cont_dict[40]], capacity), f"tx{k}{i}{j}{40}")
				model.AddMultiplicationEquality(tx40, x[(k, i, j)], t[(k, i, j, 40)])
				tx = model.NewIntVar(0, min(empty_pend[i][j][cont_dict[20]] + empty_pend[i][j][cont_dict[40]], capacity ), f"tx{k}{i}{j}")
				model.Add(tx == tx20 + tx40)

				# For DS, sum of loaded_pend + empty_pend for 20 ft containers <= 50% of capacity.
				if stack[i][j] == 4:
					model.Add(2 * (yx20 + tx20) <= capacity)

				# if (loaded_pendency < 50 % of capacity
				# then 100 % capacity to be utilized through empty_pendency)
				# Note: For double stack, this 50 % becomes 25 %
				model.Add(yx + tx <= capacity).OnlyEnforceIf(bt[(k, i, j)]).OnlyEnforceIf(x[(k, i, j)])
				# If (i->j) is a closed_route
				# we ignore < 50 % loaded_pendency constraint on this path
				# Note: For double stack, this 50 % becomes 25 %
				if (start_loc[train], loc2) in closed_rt:
					model.Add(yx + tx <= capacity).OnlyEnforceIf(bt[(k, i, j)].Not()).OnlyEnforceIf(x[(k, i, j)])
				else:
					model.Add(yx + tx == capacity).OnlyEnforceIf(bt[(k, i, j)].Not()).OnlyEnforceIf(x[(k, i, j)])

		# Objective changed to max sum over(i, j, k) Weightage_Load_Pend * {y_{kij} * x_{kij}} + {t_{kij} * x{kij}}
		model.Maximize(Weightage_Load_Pend * cp_model.LinearExpr.Sum(load_expr) + cp_model.LinearExpr.Sum(empty_expr))

		print("Model Building Completed")
		solver = cp_model.CpSolver()
		solver.parameters.log_search_progress = False
		solver.parameters.cp_model_presolve = True
		solver.parameters.max_time_in_seconds = 120
		solver.random_seed = 42
		print("Solving Started")
		status = solver.Solve(model)
		status_dict = {
		0: "UNKNOWN: The status of the model is still unknown. A search limit has been reached before any of the statuses below could be determined.",
		1: "MODEL_INVALID: The given CpModelProto didn't pass the validation step.",
		2: "FEASIBLE: A feasible solution has been found. But the search was stopped before we could prove optimality.",
		3: "INFEASIBLE: The problem has been proven infeasible.",
		4: "OPTIMAL: An optimal feasible solution has been found."
		}
		print(status_dict[status])
		if status == 2 or status == 4:
			print(f"Objective = {solver.ObjectiveValue()}")
			cols = ['Train', 'Source', 'Destination', 'closed_rt (i -> j)', 'Loaded_Pend_Cleared_20', 'Pendency_20', 'Loaded_Pend_Cleared_40', 'Pendency_40',
					'bt', 'Empty_Pend_Cleared_20', 'Empties_20','Empty_Pend_Cleared_40', 'Empties_40', 'Utilized_Space', 'Capacity', 'Wagons', 'Stack', 'Base_Loc', 'Dist (i -> j)',
					'Dist(j -> base)', 'Dist(i -> base)', 'Dist_Travelled', 'Rem_Dist_Km', 'Total_Time_Mins', 'Rem_Time_Mins', 'TXR_Flag']
			df_out = pd.DataFrame(columns = cols)
			trains_not_scheduled = []
			start_loc_not_scheduled = []
			pend_train_not_scheduled = []
			eta_train_not_scheduled = []
			closed_route_train = []
			for train in Trains:
				flag = 0
				loc1 = start_loc[train]
				for loc2 in locs:
					if loc1 == loc2:
						continue
					k, i, j = train_dict[train], loc_dict[loc1], loc_dict[loc2]
					capacity = wagons[train_dict[train]] * stack[i][j]
					utilized = solver.Value(y[(k, i, j, 20)]) + solver.Value(t[(k, i, j, 20)]) + solver.Value(y[(k, i, j, 40)]) + solver.Value(t[(k, i, j, 40)])
					close_rt_flag = 0
					if (loc1, loc2) in closed_rt:
						close_rt_flag = 1
					if solver.Value(x[(k, i, j)]):
						flag = 1
						if loc2 == base_depot_dict[train]:
							base_idx = loc_dict[base_depot_dict[train]]
							txr_flag = 0
							if (dist[i][j] + dist[j][base_idx]) > Rem_Dist[k] or (time[i][j] + time[j][base_idx]) > Rem_Time[k]:
								txr_flag = 1
							data = [train, loc1, loc2, close_rt_flag, solver.Value(y[(k, i, j, 20)]), pend[i][j][cont_dict[20]],
									solver.Value(y[(k, i, j, 40)]), pend[i][j][cont_dict[40]], solver.Value(bt[(k, i, j)]), solver.Value(t[(k, i, j, 20)]), empty_pend[i][j][cont_dict[20]],
									solver.Value(t[(k, i, j, 40)]), empty_pend[i][j][cont_dict[40]], utilized, capacity, wagons[train_dict[train]],
									stack[i][j], base_depot_dict[train], 0, 0, dist[i][base_idx], dist[i][base_idx], Rem_Dist[k], time[i][base_idx], Rem_Time[k], txr_flag]
							new_row = pd.DataFrame([data], columns = cols)
							df_out = pd.concat([df_out, new_row], ignore_index = True)
						else:
							base_idx = loc_dict[base_depot_dict[train]]
							txr_flag = 0
							if (dist[i][j] + dist[j][base_idx]) > Rem_Dist[k] or (time[i][j] + time[j][base_idx]) > Rem_Time[k]:
								txr_flag = 1
							data = [train, loc1, loc2, close_rt_flag, solver.Value(y[(k, i, j, 20)]), pend[i][j][cont_dict[20]],
									solver.Value(y[(k, i, j, 40)]), pend[i][j][cont_dict[40]], solver.Value(bt[(k, i, j)]), solver.Value(t[(k, i, j, 20)]), empty_pend[i][j][cont_dict[20]],
									solver.Value(t[(k, i, j, 40)]), empty_pend[i][j][cont_dict[40]], utilized, capacity, wagons[train_dict[train]], stack[i][j],
									base_depot_dict[train], dist[i][j], dist[j][base_idx], 0, dist[i][j] + dist[j][base_idx], Rem_Dist[k],
									time[i][j] + time[j][base_idx], Rem_Time[k], txr_flag]
							new_row = pd.DataFrame([data], columns = cols)
							df_out = pd.concat([df_out, new_row], ignore_index = True)
				if flag == 0:
					trains_not_scheduled.append(train)
					start_loc_not_scheduled.append(start_loc[train])
					pend_train_not_scheduled.append((pend_sums[loc_dict[start_loc[train]]]))
					eta_train_not_scheduled.append(Eta[train_dict[train]])
					closed_route_train.append(closed_rt_train[k])
			df_out_1 = pd.DataFrame()
			df_out_1['Trains_Not_Scheduled'] = trains_not_scheduled
			df_out_1['closed_rt_train'] = closed_route_train
			df_out_1['Start_Loc'] = start_loc_not_scheduled
			df_out_1['Total Pendency'] = pend_train_not_scheduled
			df_out_1['ETA'] = eta_train_not_scheduled
			# df_out.to_csv('./05122023_output_empty_pend.csv', index = False)
			# df_out_1.to_csv('./05122023_unscheduled_trains_empty_pend.csv', index = False)
			print(f"Trains_Scheduled = {len(Trains) - len(trains_not_scheduled)}/{len(Trains)}")
			pend_df_upd = get_pend_updated(pend_df, df_out)
			# pend_df_upd = pd.DataFrame()
			return df_out, df_out_1, pend_df_upd

	def nearest_lower_even(num):
		return 2 * (num // 2)

	def schedule_zero_pend_train(unscheduled_trains, locs, loc_dict, dist, time, stack,
								 Trains, train_dict, Rem_Dist, Rem_Time, start_loc,
								 wagons, base_loc, base_depot_dict, updated_pend, pend_sums_upd, empty_pend_upd):
		z = {}
		for train in unscheduled_trains:
			loc1 = start_loc[train]
			i, k = loc_dict[loc1], train_dict[train]
			for loc2 in locs:
				for loc3 in locs:
					j, m = loc_dict[loc2], loc_dict[loc3]
					z[(k, i, j, m)] = 0

		train_upd_pend_cleared_20 = {}
		train_upd_pend_cleared_40 = {}
		train_upd_pend_20 = {}
		train_upd_pend_40 = {}
		train_upd_emp_cleared_20 = {}
		train_upd_emp_cleared_40 = {}
		train_upd_emp_pend_20 = {}
		train_upd_emp_pend_40 = {}
		train_upd_pend_cleared = {}
		train_upd_emp_cleared = {}
		train_capacity = {}
		train_total_pend_cleared = {}

		updated_pend1 = updated_pend.copy()
		empty_pend_upd1 =empty_pend_upd.copy()
		for train in unscheduled_trains:
			loc1 = start_loc[train]
			i= loc_dict[loc1]
			indices = []
			for j in range(len(locs)):
				if dist[i][j] == INF:
					continue
				indices.append(j)

			sorted_indices = sorted(indices, key=lambda j: dist[i][j])
			k = train_dict[train]
			b = loc_dict[base_depot_dict[train]]
			for j in sorted_indices:
				if (Rem_Dist[k] < dist[i][j]) or (Rem_Time[k] < time[i][j]):
					continue
				else:
					if pend_sums[j] == 0:
						#sum(pend[j])
						continue
					indices2 = [m for m in range(len(locs)) if j!= m and dist[j][m] != INF and dist[m][b] != INF]
					sorted_indices2 = sorted(indices2, key=lambda m: (2*nearest_lower_even(min((updated_pend[j][m][cont_dict[20]]+updated_pend[j][m][cont_dict[40]]), wagons[train_dict[train]] * stack[j][m])) + nearest_lower_even(min((empty_pend_upd[j][m][cont_dict[20]]+empty_pend_upd[j][m][cont_dict[40]]), wagons[train_dict[train]] * stack[j][m]))) , reverse=True)
					flag = 0
					for m in sorted_indices2:
						if stack[j][m] == 2:
							if (nearest_lower_even(min((updated_pend[j][m][cont_dict[20]]+updated_pend[j][m][cont_dict[40]]), wagons[train_dict[train]] * stack[j][m])) <  1 * wagons[train_dict[train]] * stack[j][m]) and (nearest_lower_even(min((updated_pend[j][m][cont_dict[20]]+updated_pend[j][m][cont_dict[40]]+empty_pend_upd[j][m][cont_dict[20]]+empty_pend_upd[j][m][cont_dict[40]]), wagons[train_dict[train]] * stack[j][m])) < wagons[train_dict[train]] * stack[j][m]):
								continue
							if (Rem_Dist[k] < dist[i][j] + dist[j][m] + dist[m][b]) or (Rem_Time[k] < time[i][j] + time[j][m] + time[m][b]):
								continue
							z[(k, i, j, m)] = 1
							train_capacity[k] = wagons[train_dict[train]] * stack[j][m]
							train_upd_pend_cleared_40[k] = nearest_lower_even(min(updated_pend[j][m][cont_dict[40]], wagons[train_dict[train]] * stack[j][m]))
							train_upd_pend_cleared_20[k] = nearest_lower_even(min(updated_pend[j][m][cont_dict[20]], (wagons[train_dict[train]] * stack[j][m] - train_upd_pend_cleared_40[k])))
							train_upd_emp_cleared_40[k] = nearest_lower_even(min(train_capacity[k] - train_upd_pend_cleared_20[k] - train_upd_pend_cleared_40[k] , empty_pend_upd[j][m][cont_dict[40]]))
							train_upd_emp_cleared_20[k] = nearest_lower_even(min(train_capacity[k] - train_upd_pend_cleared_20[k] - train_upd_pend_cleared_40[k] - train_upd_emp_cleared_40[k] , empty_pend_upd[j][m][cont_dict[20]]))
							train_upd_pend_cleared[k] = train_upd_pend_cleared_20[k] + train_upd_pend_cleared_40[k]
							train_upd_emp_cleared[k] = train_upd_emp_cleared_40[k] + train_upd_emp_cleared_20[k]
							train_total_pend_cleared[k] = train_upd_pend_cleared[k] + train_upd_emp_cleared[k]
							train_upd_pend_20[k] = updated_pend[j][m][cont_dict[20]]
							train_upd_pend_40[k] = updated_pend[j][m][cont_dict[40]]
							updated_pend[j][m][cont_dict[20]] = updated_pend[j][m][cont_dict[20]] - train_upd_pend_cleared_20[k]
							updated_pend[j][m][cont_dict[40]] = updated_pend[j][m][cont_dict[40]] - train_upd_pend_cleared_40[k]
							train_upd_emp_pend_20[k] = empty_pend_upd[j][m][cont_dict[20]]
							train_upd_emp_pend_40[k] = empty_pend_upd[j][m][cont_dict[40]]
							empty_pend_upd[j][m][cont_dict[20]] = empty_pend_upd[j][m][cont_dict[20]] - train_upd_emp_cleared_20[k]
							empty_pend_upd[j][m][cont_dict[40]] = empty_pend_upd[j][m][cont_dict[40]] - train_upd_emp_cleared_40[k]
							flag = 1
							break
						else:
							if (nearest_lower_even(min((updated_pend[j][m][cont_dict[20]]+updated_pend[j][m][cont_dict[40]]), wagons[train_dict[train]] * stack[j][m])) <  0.25 * wagons[train_dict[train]] * stack[j][m]) and (nearest_lower_even(min((updated_pend[j][m][cont_dict[20]]+updated_pend[j][m][cont_dict[40]]+empty_pend_upd[j][m][cont_dict[20]]+empty_pend_upd[j][m][cont_dict[40]]), wagons[train_dict[train]] * stack[j][m])) < wagons[train_dict[train]] * stack[j][m]):
								continue
							if (Rem_Dist[k] < dist[i][j] + dist[j][m] + dist[m][b] ) or (Rem_Time[k] < time[i][j] + time[j][m] + time[m][b]):
								continue
							train_capacity[k] = wagons[train_dict[train]] * stack[j][m]
							train_upd_pend_cleared_40[k] = nearest_lower_even(min(updated_pend[j][m][cont_dict[40]], wagons[train_dict[train]] * stack[j][m]))
							train_upd_pend_cleared_20[k] = nearest_lower_even(min(updated_pend[j][m][cont_dict[20]], (wagons[train_dict[train]] * stack[j][m] - train_upd_pend_cleared_40[k]),wagons[train_dict[train]] * stack[j][m] *0.5))
							train_upd_emp_cleared_40[k] = nearest_lower_even(min(train_capacity[k] - train_upd_pend_cleared_20[k] - train_upd_pend_cleared_40[k] , empty_pend_upd[j][m][cont_dict[40]]))
							train_upd_emp_cleared_20[k] = nearest_lower_even(min(train_capacity[k] - train_upd_pend_cleared_20[k] - train_upd_pend_cleared_40[k] - train_upd_emp_cleared_40[k] , empty_pend_upd[j][m][cont_dict[20]],wagons[train_dict[train]] * stack[j][m] *0.5))
							train_upd_pend_cleared[k] = train_upd_pend_cleared_20[k] + train_upd_pend_cleared_40[k]
							train_upd_emp_cleared[k] = train_upd_emp_cleared_40[k] + train_upd_emp_cleared_20[k]
							train_total_pend_cleared[k] = train_upd_pend_cleared[k] + train_upd_emp_cleared[k]
							if (train_upd_pend_cleared_40[k] + train_upd_pend_cleared_20[k] < 0.5*wagons[train_dict[train]] * stack[j][m]) and (train_total_pend_cleared[k] < wagons[train_dict[train]] * stack[j][m]):
								continue
							z[(k, i, j, m)] = 1
							train_upd_pend_20[k] = updated_pend[j][m][cont_dict[20]]
							train_upd_pend_40[k] = updated_pend[j][m][cont_dict[40]]
							updated_pend[j][m][cont_dict[20]] = updated_pend[j][m][cont_dict[20]] - train_upd_pend_cleared_20[k]
							updated_pend[j][m][cont_dict[40]] = updated_pend[j][m][cont_dict[40]] - train_upd_pend_cleared_40[k]
							train_upd_emp_pend_20[k] = empty_pend_upd[j][m][cont_dict[20]]
							train_upd_emp_pend_40[k] = empty_pend_upd[j][m][cont_dict[40]]
							empty_pend_upd[j][m][cont_dict[20]] = empty_pend_upd[j][m][cont_dict[20]] - train_upd_emp_cleared_20[k]
							empty_pend_upd[j][m][cont_dict[40]] = empty_pend_upd[j][m][cont_dict[40]] - train_upd_emp_cleared_40[k]
							flag = 1
							break

				if flag:
					break


		cols = ['Train', 'Source(i)', 'j', 'Txr_Flag', 'Loaded_pend_cleared_20(i -> j)','Loaded_pend_cleared_40(i -> j)', 'Emp_pend_cleared_20(i -> j)','Emp_pend_cleared_40(i -> j)','Total_pend_cleared(i -> j)','Capacity:(i->j)', 'Loaded_pend_20(i -> j)','Loaded_pend_40(i -> j)', 'Emp_pend_20(i -> j)','Emp_pend_40(i -> j)', 'm', 'Loaded_pend_cleared_20(j -> m)', 'Loaded_pend_cleared_40(j -> m)', 'Emp_pend_cleared_20(j -> m)','Emp_pend_cleared_40(j -> m)','Total_Loaded_pend_cleared(j -> m)','Total_Emp_pend_cleared(j -> m)','Total_pend_cleared(j -> m)','Capacity:(j->m)', 'Loaded_pend_20(j -> m)','Loaded_pend_40(j -> m)', 'Emp_pend(j -> m)_20','Emp_pend(j -> m)_40']
		df_zero_pend_trains = pd.DataFrame(columns = cols)
		for train in unscheduled_trains:
			loc1 = start_loc[train]
			i, k, b = loc_dict[loc1], train_dict[train], loc_dict[base_depot_dict[train]]
			flag = 0
			for loc2 in locs:
				for loc3 in locs:
					j, m = loc_dict[loc2], loc_dict[loc3]
					if z[(k, i, j, m)] == 1:
						data = [train, loc1, loc2, 0, updated_pend1[i][j][cont_dict[20]], updated_pend1[i][j][cont_dict[40]],  empty_pend_upd1[i][j][cont_dict[20]], empty_pend_upd1[i][j][cont_dict[40]], (updated_pend1[i][j][cont_dict[20]] + updated_pend1[i][j][cont_dict[40]] + empty_pend_upd1[i][j][cont_dict[20]] + empty_pend_upd1[i][j][cont_dict[40]]) ,wagons[train_dict[train]]* stack[i][j], updated_pend1[i][j][cont_dict[20]], updated_pend1[i][j][cont_dict[40]],  empty_pend_upd1[i][j][cont_dict[20]], empty_pend_upd1[i][j][cont_dict[40]],  loc3, train_upd_pend_cleared_20[k],train_upd_pend_cleared_40[k] , train_upd_emp_cleared_20[k], train_upd_emp_cleared_40[k] , train_upd_pend_cleared[k],train_upd_emp_cleared[k] ,train_total_pend_cleared[k], wagons[train_dict[train]] * stack[j][m], train_upd_pend_20[k],train_upd_pend_40[k],  train_upd_emp_pend_20[k], train_upd_emp_pend_40[k]]
						new_row =pd.DataFrame([data], columns = cols)
						df_zero_pend_trains = pd.concat([df_zero_pend_trains, new_row], ignore_index = True)
						flag = 1
						break
				if flag:
					break
			if flag == 0:
				txr_flag = 0
				if dist[i][b] > Rem_Dist[k] or time[i][b] > Rem_Time[k]:
					txr_flag = 1
				park = f"{base_depot_dict[train]}"
				loaded_pend_cleared_ij_20 = ''
				loaded_pend_cleared_ij_40 = ''
				empty_pend_cleared_ij_20 = ''
				empty_pend_cleared_ij_40 = ''
				total_pend_cleared_ij = ''
				loaded_pend_ij_20 = ''
				loaded_pend_ij_40 = ''
				empty_pend_ij_20 = ''
				empty_pend_ij_40 = ''
				train_cap_ij = ''
				if not txr_flag:
					park = "(PARK)"
				if txr_flag and (dist[i][b] == INF or time[i][b] == INF):
					park = "(PARK_NO_ROUTE_BASE)"
				if txr_flag == 1 and park != "(PARK_NO_ROUTE_BASE)" and i!= b:
					train_capacity[k] = wagons[train_dict[train]] * stack[i][b]
					train_cap_ij = train_capacity[k]
					if stack[i][b] == 2:
						train_upd_pend_cleared_40[k] = nearest_lower_even(min(updated_pend[i][b][cont_dict[40]], wagons[train_dict[train]] * stack[i][b]))
						train_upd_pend_cleared_20[k] = nearest_lower_even(min(updated_pend[i][b][cont_dict[20]], wagons[train_dict[train]] * stack[i][b] - train_upd_pend_cleared_40[k]))
						train_upd_emp_cleared_40[k] = nearest_lower_even(min(train_capacity[k] - train_upd_pend_cleared_20[k] -train_upd_pend_cleared_40[k] , empty_pend_upd[i][b][cont_dict[40]] ))
						train_upd_emp_cleared_20[k] = nearest_lower_even(min(train_capacity[k] - train_upd_pend_cleared_20[k] -train_upd_pend_cleared_40[k] -train_upd_emp_cleared_40[k]  , empty_pend_upd[i][b][cont_dict[20]] ))
						train_total_pend_cleared[k] = train_upd_pend_cleared_20[k] +train_upd_pend_cleared_40[k] + train_upd_emp_cleared_20[k] + train_upd_emp_cleared_40[k]
					else:
						train_upd_pend_cleared_40[k] = nearest_lower_even(min(updated_pend[i][b][cont_dict[40]], wagons[train_dict[train]] * stack[i][b]))
						train_upd_pend_cleared_20[k] = nearest_lower_even(min(updated_pend[i][b][cont_dict[20]], wagons[train_dict[train]] * stack[i][b] - train_upd_pend_cleared_40[k] , 0.5*wagons[train_dict[train]]*stack[i][b]))
						train_upd_emp_cleared_40[k] = nearest_lower_even(min(train_capacity[k] - train_upd_pend_cleared_20[k] -train_upd_pend_cleared_40[k] , empty_pend_upd[i][b][cont_dict[40]] ))
						train_upd_emp_cleared_20[k] = nearest_lower_even(min(train_capacity[k] - train_upd_pend_cleared_20[k] -train_upd_pend_cleared_40[k] -train_upd_emp_cleared_40[k]  , empty_pend_upd[i][b][cont_dict[20]] ,0.5*wagons[train_dict[train]]*stack[i][b] ))
						train_total_pend_cleared[k] = train_upd_pend_cleared_20[k] +train_upd_pend_cleared_40[k] + train_upd_emp_cleared_20[k] + train_upd_emp_cleared_40[k]
					loaded_pend_cleared_ij_40 = train_upd_pend_cleared_40[k]
					loaded_pend_cleared_ij_20 = train_upd_pend_cleared_20[k]
					empty_pend_cleared_ij_40 = train_upd_emp_cleared_40[k]
					empty_pend_cleared_ij_20 = train_upd_emp_cleared_20[k]
					total_pend_cleared_ij = loaded_pend_cleared_ij_20 +loaded_pend_cleared_ij_40 + empty_pend_cleared_ij_40 +empty_pend_cleared_ij_20
					train_upd_pend_20[k] = updated_pend[i][b][cont_dict[20]]
					loaded_pend_ij_20 = train_upd_pend_20[k]
					updated_pend[i][b][cont_dict[20]] = updated_pend[i][b][cont_dict[20]] - train_upd_pend_cleared_20[k]
					train_upd_pend_40[k] = updated_pend[i][b][cont_dict[40]]
					loaded_pend_ij_40 = train_upd_pend_40[k]
					updated_pend[i][b][cont_dict[40]] = updated_pend[i][b][cont_dict[40]] - train_upd_pend_cleared_40[k]
					train_upd_emp_pend_20[k] = empty_pend_upd[i][b][cont_dict[20]]
					empty_pend_ij_20 = train_upd_emp_pend_20[k]
					empty_pend_upd[i][b][cont_dict[20]] = empty_pend_upd[i][b][cont_dict[20]] - train_upd_emp_cleared_20[k]
					train_upd_emp_pend_40[k] = empty_pend_upd[i][b][cont_dict[40]]
					empty_pend_ij_40 = train_upd_emp_pend_40[k]
					empty_pend_upd[i][b][cont_dict[40]] = empty_pend_upd[i][b][cont_dict[40]] - train_upd_emp_cleared_40[k]

				data = [train, loc1, park, txr_flag, loaded_pend_cleared_ij_20, loaded_pend_cleared_ij_40, empty_pend_cleared_ij_20,empty_pend_cleared_ij_40, total_pend_cleared_ij, train_cap_ij,  loaded_pend_ij_20, loaded_pend_ij_40, empty_pend_ij_20, empty_pend_ij_40, '', '', '','','','','','','','','','','']
				new_row =pd.DataFrame([data], columns = cols)
				df_zero_pend_trains = pd.concat([df_zero_pend_trains, new_row], ignore_index = True)
				z[(k, i, j, m)] = 1

		df_zero_pend_trains_out = df_zero_pend_trains.rename(columns={'j':'Source','m':'Destination','Loaded_pend_cleared_20(j -> m)':'Loaded_Pend_Cleared_20','Loaded_pend_cleared_40(j -> m)':'Loaded_Pend_Cleared_40','Emp_pend_cleared_20(j -> m)':'Empty_Pend_Cleared_20','Emp_pend_cleared_40(j -> m)':'Empty_Pend_Cleared_40'})
		pend_df_upd_intmd = get_pend_updated(pend_df_upd, df_zero_pend_trains_out)

		# print(pend_df_upd_intmd.head())
		df_zero_pend_trains_intmd = df_zero_pend_trains.rename(columns={'Source(i)':'Source','j':'Destination','Loaded_pend_cleared_20(j -> m)':'Loaded_Pend_Cleared_20','Loaded_pend_cleared_40(j -> m)':'Loaded_Pend_Cleared_40','Emp_pend_cleared_20(j -> m)':'Empty_Pend_Cleared_20','Emp_pend_cleared_40(j -> m)':'Empty_Pend_Cleared_40'})
		pend_df_upd_round2 = get_pend_updated(pend_df_upd_intmd, df_zero_pend_trains_intmd)

		return df_zero_pend_trains, pend_df_upd_round2

	dist_time_df, pend_df, train_curr_plan_df = data_prep(df_dist_time, df_pendency, df_train_curr_plan)

	locs, loc_dict = get_loc(dist_time_df, pend_df, train_curr_plan_df) # locations are indexed from [0, num_loc - 1]
	Trains, train_dict, Rem_Dist, Rem_Time, start_loc, wagons, Eta = get_trains_rem_dist_time(train_curr_plan_df) # trains are indexed from [0, num_trains - 1]
	base_loc, base_depot_dict = get_base_depot(train_curr_plan_df, Trains, train_dict)
	dist, time, stack, closed_rt, closed_rt_train = get_dist_time_stack(dist_time_df, len(locs), loc_dict, locs, base_loc, Trains, train_dict, start_loc)
	pend, pend_sums, empty_pend = get_pendency(pend_df, len(locs), loc_dict)

	num_trains, num_stations, num_base, num_loc = len(Trains), len(locs) - len(base_loc), len(base_loc), len(locs)
	print(f"Total_Trains = {len(Trains)}: Total_Stations = {len(locs) - len(base_loc)}: Total_Base_Locs = {len(base_loc)}")
	print("Model_Building_Started")

	# Solving the original model
	df_out, df_out_1, pend_df_upd = optimize(num_trains, num_stations, num_base, num_loc, locs, Trains, train_dict,
				dist, time, Rem_Dist, Rem_Time, loc_dict, base_loc, base_depot_dict, pend, empty_pend, stack, start_loc,
				pend_sums, wagons, Eta, closed_rt, closed_rt_train, dist_time_df, pend_df, train_curr_plan_df)



    ################################################################################
	# TRANSHIPMENT - HUB PENDENCY
	################################################################################

	# run the first optimizer (done by this step)
	# find out rakes which carry less than full capacity for which hub route is present
	# unschedule those rakes, add back the pendency for those routes to output pendency data of first optimizer
	# find out routes which are common between pendency_data and via_rt data (where route type in via_rt is transshipment),
	# group by and modify those as origin and via instead of origin and destination

	# To handle - Transhipment Hub Pendency

	# rakes where full capacity isn't utlized

	unscheduled_trains_lesscap = []
	# lesscap_trains dictionary includes all the rakes where the capacity is not fully utilized and all the rakes from df_out_1 (unscheduled rakes)
	lesscap_trains = (df_out[df_out.Utilized_Space != df_out.Capacity].set_index('Train')['Source'].to_dict())

	via_rt_origins = via_rt.origin_code.unique().tolist()
	# unscheduling rakes which are at via-rt origins only
	for i in lesscap_trains:
	  if lesscap_trains[i] in via_rt_origins:
		unscheduled_trains_lesscap.append(i)

	# if any train with unutilized capacity is found then only proceed
	if len(unscheduled_trains_lesscap) > 0:

	  # df_out of all less capacity trains
	  df_out_x = df_out[df_out.Train.isin(unscheduled_trains_lesscap)].reset_index(drop = True)

	  ##############################################################################
	  # PENDENCY_DATA DATA
	  ##############################################################################

	  # add back the pendency for routes found above (where pendency_data is output of pendency_data after first optimizer is run)
	  for i in range(df_out_x.shape[0]):
		source = df_out_x.loc[i,"Source"]
		destination = df_out_x.loc[i,"Destination"]
		loaded_pend_cleared_20 = df_out_x.loc[i,"Loaded_Pend_Cleared_20"]
		loaded_pend_cleared_40 = df_out_x.loc[i,"Loaded_Pend_Cleared_40"]
		emp_pend_cleared_20 = df_out_x.loc[i,"Empty_Pend_Cleared_20"]
		emp_pend_cleared_40 = df_out_x.loc[i,"Empty_Pend_Cleared_40"]
		# pend_df_upd is the updated pendency after first optimizer is run
		for j in range(pend_df_upd.shape[0]):
		  if ((pend_df_upd.loc[j,"Source"] == source) & (pend_df_upd.loc[j,"Destination"] == destination)) :
			if pend_df_upd.loc[j,"cont_size"] == 20.0:
			  pend_df_upd.loc[j,"Pendency"] += int(loaded_pend_cleared_20)               # add back loaded pendency
			  pend_df_upd.loc[j,"empty_pendency"] += int(emp_pend_cleared_20)            # add back empty pendency
			if pend_df_upd.loc[j,"cont_size"] == 40.0:
			  pend_df_upd.loc[j,"Pendency"] += int(loaded_pend_cleared_40)               # add back loaded pendency
			  pend_df_upd.loc[j,"empty_pendency"] += int(emp_pend_cleared_40)            # add back empty pendency

	  # unschedule rakes where full capacity isn't utilized are removed from scheduled trains list
	  df_out = df_out[~df_out.Train.isin(unscheduled_trains_lesscap)].reset_index(drop = True)

	  schedule1 = df_out.copy()

	  # delete the previous outputs
	  del df_out

	  # find out routes which are common between pendency_data and via_rt data (where route type in via_rt is Transhipment-Hub),

	  pend_df_upd_x = pend_df_upd.merge(via_rt[via_rt.route_type != "Maintenance"][["origin_code","destination_code","via","route_type"]]
										, left_on = ["Source","Destination"]
										, right_on = ["origin_code","destination_code"]
										, how = 'left')

	  pend_df_upd_x.via = pend_df_upd_x.via.fillna(pend_df_upd_x.Destination)

	  # group by and modify those as origin and via instead of origin and destination for "Transhipment-Hub" type via routes
	  for i in range(pend_df_upd_x.shape[0]):
		if pend_df_upd_x.loc[i,"route_type"] == "Transhipment-Indirect":
		  continue
		pend_df_upd_x.loc[i,"Destination"] = pend_df_upd_x.loc[i,"via"]

	  pend_df_upd_x = pend_df_upd_x.groupby(["Source","Destination","cont_size"])[['Pendency','empty_pendency']].sum().reset_index()

	  new_df_pendency = (df_pendency[["Source","Destination","odr_days","cont_size"]].merge(pend_df_upd_x[["Pendency","empty_pendency","Source","Destination","cont_size"]],
																			  on = ["Source","Destination","cont_size"],
																			  how = 'inner'))[["Pendency","empty_pendency","Source","Destination","odr_days","cont_size"]]

	  del df_pendency

	  df_pendency = new_df_pendency.reset_index(drop = True)

	  ##############################################################################
	  # DIST TIME DATA
	  ##############################################################################
	  df_dist_time = df_dist_time.reset_index(drop = True)
	  ##############################################################################
	  # TRAIN CURRENT STATUS DATA
	  ##############################################################################
	  lcap_df_train_curr_plan = train_current_status[train_current_status.rake_name.isin(list(df_out_1.Trains_Not_Scheduled.unique()) + unscheduled_trains_lesscap)]
	  # delete previous outputs
	  del df_out_1
	  del df_train_curr_plan
	  df_train_curr_plan = lcap_df_train_curr_plan.reset_index(drop = True)
	  ##############################################################################
	  # OPTIMIZATION
	  ##############################################################################
	  dist_time_df, pend_df, train_curr_plan_df = data_prep(df_dist_time, df_pendency, df_train_curr_plan)

	  locs, loc_dict = get_loc(dist_time_df, pend_df, train_curr_plan_df) # locations are indexed from [0, num_loc - 1]
	  Trains, train_dict, Rem_Dist, Rem_Time, start_loc, wagons, Eta = get_trains_rem_dist_time(train_curr_plan_df) # trains are indexed from [0, num_trains - 1]
	  base_loc, base_depot_dict = get_base_depot(train_curr_plan_df, Trains, train_dict)
	  dist, time, stack, closed_rt, closed_rt_train = get_dist_time_stack(dist_time_df, len(locs), loc_dict, locs, base_loc, Trains, train_dict, start_loc)
	  pend, pend_sums, empty_pend = get_pendency(pend_df, len(locs), loc_dict)

	  num_trains, num_stations, num_base, num_loc = len(Trains), len(locs) - len(base_loc), len(base_loc), len(locs)
	  print(f"Total_Trains = {len(Trains)}: Total_Stations = {len(locs) - len(base_loc)}: Total_Base_Locs = {len(base_loc)}")
	  print("Model_Building_Started")

	  # Solving the original model
	  df_out, df_out_1, pend_df_upd = optimize(num_trains, num_stations, num_base, num_loc, locs, Trains, train_dict,
			  dist, time, Rem_Dist, Rem_Time, loc_dict, base_loc, base_depot_dict, pend, empty_pend, stack, start_loc,
			  pend_sums, wagons, Eta, closed_rt, closed_rt_train, dist_time_df, pend_df, train_curr_plan_df)

	  schedule1 = pd.concat([schedule1, df_out]).reset_index(drop = True)    # schedule1.append(df_out).reset_index(drop = True)

	else:

	  schedule1 = df_out.copy()
	  df_out_1 = df_out_1.copy()


	def update_schedules(df):

	  # for unscheduled trains (df_out_1 trains)
	  if 'm' in df.columns:
		df.rename(columns = {'Train':'rake_name', 'Source(i)':'origin_code', 'j':'destination_code',
							 'Txr_Flag':'TXR_Flag',
							 'm':'destination_code2',
							 'Loaded_pend_cleared_20(j -> m)':'loaded_TEUs_evacuated2_20',
							 'Loaded_pend_cleared_40(j -> m)':'loaded_TEUs_evacuated2_40',
							 'Emp_pend_cleared_20(j -> m)':'empty_TEUs_evacuated2_20',
							 'Emp_pend_cleared_40(j -> m)':'empty_TEUs_evacuated2_40',
							 'Total_Loaded_pend_cleared(j -> m)':'Total_loaded_evacuated',
							 'Total_Emp_pend_cleared(j -> m)':'Total_empties_evacuated',
							 'Total_pend_cleared(j -> m)':'Total_TEUs_evacuated'
						   }, inplace = True)

		df = df.merge(train_curr_plan_df[["Train","base_depot_code","unts"]], left_on = ["rake_name"], right_on = ["Train"], how = "left")
		df = df.merge(overall_pendency_data,
					  left_on = ["destination_code","destination_code2"],
					  right_on = ["booking_location_code","to_terminal_code"],
					  how = 'left')
		for idx in range(df.shape[0]):
		  # if txr flag is 1 then add 'TXR' at the end
		  if df.loc[idx,"TXR_Flag"] == 1:
			df.loc[idx,"destination_code"] = df.loc[idx,"destination_code"] + "-(TXR)"
		  if ((df.loc[idx,"TXR_Flag"] == 0) & (df.loc[idx,"destination_code2"] != "")):
			  df.loc[idx,"destination_code"] = df.loc[idx,"destination_code"] + "-" + df.loc[idx,"destination_code2"]

		df.rename(columns =  {'loaded_TEUs_evacuated2_20':'loaded_TEUs_evacuated_20',
							  'loaded_TEUs_evacuated2_40':'loaded_TEUs_evacuated_40',
							  'empty_TEUs_evacuated2_20' : 'empty_TEUs_evacuated_20',
							  'empty_TEUs_evacuated2_40' : 'empty_TEUs_evacuated_40',
							  'stack2':'stack','unts':'wagons'
									}, inplace = True)
		df = df[["rake_name","origin_code","destination_code","loaded_TEUs_evacuated_20",
				 "loaded_TEUs_evacuated_40","empty_TEUs_evacuated_20","empty_TEUs_evacuated_40","route","pendency","empty_pendency"]]
		cols_type_int = ['loaded_TEUs_evacuated_20','loaded_TEUs_evacuated_40','empty_TEUs_evacuated_20','empty_TEUs_evacuated_40']
		df[cols_type_int] = df[cols_type_int].replace('', 0)

	  else:
		# for scheduled trains (df_out)
		df.rename(columns = {'Train':'rake_name', 'Source':'origin_code', 'Destination':'destination_code',
							'closed_rt (i -> j)':'closed_rt',
							'Loaded_Pend_Cleared_20':'loaded_TEUs_evacuated_20',
							'Pendency_20':'pendency_loaded20',
							'Loaded_Pend_Cleared_40':'loaded_TEUs_evacuated_40',
							'Pendency_40':'pendency_loaded40',
							'Empty_Pend_Cleared_20':'empty_TEUs_evacuated_20',
							'Empties_20':'pendency_empties20',
							'Empty_Pend_Cleared_40':'empty_TEUs_evacuated_40',
							'Empties_40':'pendency_empties40',
							'Wagons':'wagons', 'Stack':'stack', 'Base_Loc':'base_location_code'
							}, inplace = True)

		df = df.merge(overall_pendency_data,
					  left_on = ["origin_code","destination_code"],
					  right_on = ["booking_location_code","to_terminal_code"],
					  how = 'left')

		for idx in range(df.shape[0]):
		  # special case for closed route - for closed route the rake always goes back to predefined destination
		  # for such routes, if closed_rt = 1 and txr_flag = 1 then print predefined destination + base location + txr
		  if ((df.loc[idx,'closed_rt'] == 1) & (df.loc[idx,'TXR_Flag'] == 1)):
			df.loc[idx,"destination_code"] = df.loc[idx,"destination_code"] + "-" + df.loc[idx,"base_location_code"] + "-(TXR)"
		  # if not a closed_rt and txr_flag = 1 then print destination to txr as destination is base in such cases already
		  elif ((df.loc[idx,'closed_rt'] == 0) & (df.loc[idx,'TXR_Flag'] == 1)):
			df.loc[idx,"destination_code"] = df.loc[idx,"destination_code"] + "-(TXR)"

		df = df[["rake_name","origin_code","destination_code",
				 "loaded_TEUs_evacuated_20",
				 "loaded_TEUs_evacuated_40",
				 "empty_TEUs_evacuated_20",
				 "empty_TEUs_evacuated_40","route","pendency","empty_pendency"]].reset_index(drop = True)

	  return df

	schedule1 = update_schedules(schedule1)

    ################################################################################################################################################################################################################################################
	# Unscheduled rakes from above
	################################################################################################################################################################################################################################################

	# Considering all unscheduled trains for now
	df_out_1['ETA'] = pd.to_datetime(df_out_1['ETA'])
	df_out_1=df_out_1.sort_values(by = 'ETA')
	unscheduled_trains = df_out_1['Trains_Not_Scheduled'].tolist()

	updated_pend, pend_sums_upd, empty_pend_upd = get_pendency(pend_df_upd, len(locs), loc_dict)

	print("Scheduling Unscheduled Trains")

	# Going train by train
	df_zero_pend_trains, pend_df_upd_round2 = schedule_zero_pend_train(unscheduled_trains, locs, loc_dict,
											dist, time, stack, Trains, train_dict, Rem_Dist,
											Rem_Time, start_loc, wagons,
											base_loc, base_depot_dict, updated_pend, pend_sums_upd, empty_pend_upd)

	print("Scheduling Unscheduled Trains Completed!")

	################################################################################
	# Scheduling
	################################################################################

	schedule3 = df_zero_pend_trains.copy().reset_index(drop = True)
	schedule3 = update_schedules(schedule3)


    schedule_mail_analysis = pd.concat([schedule1,schedule3]).reset_index(drop = True) #schedule1._append(schedule3)
	schedule_mail_analysis["next_trip"] = schedule_mail_analysis.origin_code + "-" + schedule_mail_analysis.destination_code
	cols_type_int = ['loaded_TEUs_evacuated_20','loaded_TEUs_evacuated_40','empty_TEUs_evacuated_20','empty_TEUs_evacuated_40']
	schedule_mail_analysis[cols_type_int] = schedule_mail_analysis[cols_type_int].astype(int)
	train_current_status_copy1["current_trip"] = train_current_status_copy1["origin_code"] + "-" + train_current_status_copy1["destination_code"]
	schedule_mail_analysis = schedule_mail_analysis.merge(train_current_status_copy1[["rake_name","current_trip","FOIS_ETA","rake_status","base_depot_code"]],
														  on = "rake_name",
														  how = "left")
	schedule_mail_analysis.rename(columns = {"FOIS_ETA":"currtrp_ETA"}, inplace = True)
	schedule_mail_analysis = schedule_mail_analysis.sort_values(by = ["next_trip","currtrp_ETA","pendency"]).reset_index(drop = True)

	for i in range(train_current_status.shape[0]):
	  if train_current_status.loc[i,"origin_code"] == train_current_status.loc[i,"destination_code"]:
		train_current_status.loc[i,"TXR_KmsRem_CurrDest"] = train_current_status.loc[i,"TXR_Kms_Remaining"]
	schedule_mail_analysis = schedule_mail_analysis.merge(train_current_status[["rake_name",
																				"TXR_Due_Date",
																				"TXR_KmsRem_CurrDest"]],
														  left_on = ["rake_name"],
														  right_on = ["rake_name"],
														  how = 'left').reset_index(drop = True)
	schedule_mail_analysis["loaded_TEUs_evacuated"] = schedule_mail_analysis["loaded_TEUs_evacuated_20"] + schedule_mail_analysis["loaded_TEUs_evacuated_40"]
	schedule_mail_analysis["empty_TEUs_evacuated"] = schedule_mail_analysis["empty_TEUs_evacuated_20"] + schedule_mail_analysis["empty_TEUs_evacuated_40"]
	schedule_mail_analysis.drop(columns = ["loaded_TEUs_evacuated_20","loaded_TEUs_evacuated_40",
										  "empty_TEUs_evacuated_20","empty_TEUs_evacuated_40", "route"], inplace = True)
	schedule_mail_analysis

    ################################################################################################################################################################################################################################################
    # Rake Schedule after TXR
    ################################################################################################################################################################################################################################################

    ################################################################################################################################################################################################################################################
	# Rake Schedule after TXR
	################################################################################################################################################################################################################################################

	# Rake Schedule after TXR
	schedule_mail_analysis_txr = schedule_mail_analysis.copy()
	txr_trains = []
	for i in range(schedule_mail_analysis_txr.shape[0]):
	  trip = schedule_mail_analysis_txr.loc[i,"destination_code"].split("-")
	  # trip = schedule_mail_analysis_txr.loc[i,"next_trip"].split("-")
	  if "(TXR)" in trip:
		txr_trains.append(schedule_mail_analysis_txr.loc[i,"rake_name"])


	if len(txr_trains) > 0:
	  del df_out, df_out_1, pend_df_upd, df_pendency, df_train_curr_plan
	  ##############################################################################
	  # DATA PREP
	  ##############################################################################
	  # PENDENCY DATA
	  ##############################################################################
	  txr_df_pendency = pendency_data[["booking_location_code","to_terminal_code","odr_days","cont_size"]].merge(pend_df_upd_round2[["Pendency","empty_pendency","Source","Destination","cont_size"]],
																	   left_on = ["booking_location_code","to_terminal_code","cont_size"],
																	   right_on = ["Source","Destination","cont_size"],
																	   how = 'inner')[["Pendency","empty_pendency","Source","Destination","odr_days","cont_size"]]
	  df_pendency = txr_df_pendency.reset_index(drop = True)
	  ##############################################################################
	  # DIST TIME DATA
	  ##############################################################################
	  df_dist_time = df_dist_time.reset_index(drop = True)
	  ##############################################################################
	  # TRAIN CURRENT STATUS DATA
	  ##############################################################################
	  txr_df_train_curr_plan = train_current_status[train_current_status.rake_name.isin(txr_trains)]
	  txr_df_train_curr_plan.TXR_Due_Date = pd.to_datetime(now) + pd.Timedelta(hours = 12) + pd.Timedelta(hours = 30*12) #12 hours + 30 days added # needs to change later for accuracy of timing
	  txr_df_train_curr_plan.TXR_Due_Date = pd.to_datetime(txr_df_train_curr_plan['TXR_Due_Date']).dt.date
	  txr_df_train_curr_plan.TXR_Kms_Remaining = 9000.0
	  txr_df_train_curr_plan.origin_code = txr_df_train_curr_plan.base_depot_code
	  txr_df_train_curr_plan.destination_code = txr_df_train_curr_plan.base_depot_code
	  txr_df_train_curr_plan.rational_distance = 0.0
	  txr_df_train_curr_plan.TXR_KmsRem_CurrDest = 9000.0
	  df_train_curr_plan = txr_df_train_curr_plan.reset_index(drop = True)

	  ##############################################################################
	  # OPTIMIZATION
	  ##############################################################################
	  dist_time_df, pend_df, train_curr_plan_df = data_prep(df_dist_time, df_pendency, df_train_curr_plan)

	  locs, loc_dict = get_loc(dist_time_df, pend_df, train_curr_plan_df) # locations are indexed from [0, num_loc - 1]
	  Trains, train_dict, Rem_Dist, Rem_Time, start_loc, wagons, Eta = get_trains_rem_dist_time(train_curr_plan_df) # trains are indexed from [0, num_trains - 1]
	  base_loc, base_depot_dict = get_base_depot(train_curr_plan_df, Trains, train_dict)
	  dist, time, stack, closed_rt, closed_rt_train = get_dist_time_stack(dist_time_df, len(locs), loc_dict, locs, base_loc, Trains, train_dict, start_loc)
	  pend, pend_sums, empty_pend = get_pendency(pend_df, len(locs), loc_dict)

	  num_trains, num_stations, num_base, num_loc = len(Trains), len(locs) - len(base_loc), len(base_loc), len(locs)
	  print(f"Total_Trains = {len(Trains)}: Total_Stations = {len(locs) - len(base_loc)}: Total_Base_Locs = {len(base_loc)}")
	  print("Model_Building_Started")

	  # Solving the original model
	  df_out, df_out_1, pend_df_upd = optimize(num_trains, num_stations, num_base, num_loc, locs, Trains, train_dict,
			  dist, time, Rem_Dist, Rem_Time, loc_dict, base_loc, base_depot_dict, pend, empty_pend, stack, start_loc,
			  pend_sums, wagons, Eta, closed_rt, closed_rt_train, dist_time_df, pend_df, train_curr_plan_df)

	  # # Considering all unscheduled trains for now
	  df_out_1['ETA'] = pd.to_datetime(df_out_1['ETA'])
	  df_out_1=df_out_1.sort_values(by = 'ETA')
	  unscheduled_trains = df_out_1['Trains_Not_Scheduled'].tolist()

	  updated_pend, pend_sums_upd, empty_pend_upd = get_pendency(pend_df_upd, len(locs), loc_dict)

	  print("Scheduling Unscheduled Trains")

	  # # Going train by train
	  df_zero_pend_trains, pend_df_upd_round2 = schedule_zero_pend_train(unscheduled_trains, locs, loc_dict,
											  dist, time, stack, Trains, train_dict, Rem_Dist,
											  Rem_Time, start_loc, wagons,
											  base_loc, base_depot_dict, updated_pend, pend_sums_upd, empty_pend_upd)

	  print("Scheduling Unscheduled Trains Completed!")

	  # ############################################################################
	  # # Scheduling
	  # ############################################################################

	  # after txr schedule of unscheduled trains
	  del schedule3, schedule1

	  schedule1 = df_out.copy().reset_index(drop = True)
	  schedule1 = update_schedules(schedule1)

	  schedule3 = df_zero_pend_trains.copy().reset_index(drop = True)
	  schedule3 = update_schedules(schedule3)

	  schedule_mail_analysis_txr = pd.concat([schedule1,schedule3]).reset_index(drop = True)

	  schedule_mail_analysis_txr = schedule_mail_analysis.merge(schedule_mail_analysis_txr[["rake_name","destination_code"]],
																on = "rake_name",
																how = "left").reset_index(drop = True)

	  # schedule_mail_txr = schedule_mail_analysis_txr.copy().reset_index(drop = True)

	  # Add the destination generated after train's txr is complete
	  for i in range(schedule_mail_analysis_txr.shape[0]):
		if pd.isna(schedule_mail_analysis_txr.destination_code_y[i]) == True:
		  continue
		schedule_mail_analysis_txr.next_trip[i] = schedule_mail_analysis_txr.next_trip[i] + "-" + schedule_mail_analysis_txr.destination_code_y[i]

	  schedule_mail_analysis_txr.drop(columns = ["destination_code_x","destination_code_y"], inplace = True)

	else:
	  schedule_mail_analysis_txr = schedule_mail_analysis.copy().reset_index(drop = True)

	################################################################################################################################################################################################################################################
	# Via route terminals (No direct route between destination and base location)
	################################################################################################################################################################################################################################################

	# Function to update via-route terminals
	def update_via_rt_terminals(df_schedule,df_via_rt,row_trip):
	  for i in range(df_schedule.shape[0]):
		trip = df_schedule.loc[i,row_trip]
		df_via_rt1 = df_via_rt[df_via_rt.route_type == "Maintenance"].reset_index(drop = True)
		for j in range(df_via_rt1.shape[0]):
		  bad_via_trip = df_via_rt1.loc[j,"route"]
		  good_via_trip = df_via_rt1.loc[j,"via_route"]
		  if bad_via_trip in trip:
			trip = trip.replace(bad_via_trip,good_via_trip)
		df_schedule.loc[i,row_trip] = trip
	  return df_schedule

	schedule_mail = update_via_rt_terminals(schedule_mail_analysis_txr, via_rt, 'next_trip')

	# Function to remove consecutive duplicates of terminals from trip names
	def remove_duplicate_terminals_from_trips(df, row):
	  for idx in range(df.shape[0]):
		# df2 = df.copy()
		L = df.loc[idx,row].split('-')
		df.loc[idx,row] = '-'.join([key for key, _group in groupby(L)])
	  return df

	schedule_mail = remove_duplicate_terminals_from_trips(schedule_mail, 'next_trip')
	schedule_mail = remove_duplicate_terminals_from_trips(schedule_mail, 'current_trip')


	schedule_mail

    # Once a train is scheduled, the pendency should be updated for that route in the output.
	def update_pendency_in_ouput(df1, row, row2):
	  all_next_trip = list(df1.next_trip.unique())
	  df = pd.DataFrame()
	  for i in (all_next_trip):
		filt_next_trip = df1[df1.next_trip == i].reset_index(drop = True)
		pend_next_trip = filt_next_trip[row][0]
		for i in range(filt_next_trip.shape[0]-1):
		  filt_next_trip.loc[i,row] = pend_next_trip
		  filt_next_trip.loc[i+1,row] = pend_next_trip - filt_next_trip.loc[i,row2]
		  pend_next_trip = filt_next_trip.loc[i+1,row]
		df = df._append(filt_next_trip).reset_index(drop = True)
	  return df

	final_df = update_pendency_in_ouput(schedule_mail, 'pendency','loaded_TEUs_evacuated')
	final_df = update_pendency_in_ouput(final_df, 'empty_pendency','empty_TEUs_evacuated')
	final_df.pendency = final_df.pendency.fillna(0)
	final_df.empty_pendency = final_df.empty_pendency.fillna(0)
	# final_df.drop(columns = ["loaded_TEUs_evacuated_20","loaded_TEUs_evacuated_40","empty_TEUs_evacuated_20","empty_TEUs_evacuated_40"], inplace = True)
	final_df["total_pendency"] = final_df["pendency"] + final_df["empty_pendency"]
	final_df["TEUs_evacuated"] = final_df["loaded_TEUs_evacuated"] + final_df["empty_TEUs_evacuated"]
	final_df.drop(columns = ["empty_pendency","pendency"], inplace = True)
	final_df.rename(columns = {"total_pendency":"pendency"}, inplace = True)
	final_df.pendency = final_df.pendency.fillna(0)
	final_df = final_df[["rake_name","rake_status","currtrp_ETA",
								   "current_trip","next_trip","pendency",
								   "TEUs_evacuated","TXR_Due_Date","TXR_KmsRem_CurrDest",
								   "base_depot_code"]].reset_index(drop = True)

	final_df

    data_df = final_df.copy()
    del schedule_mail_analysis
    schedule_mail_analysis = final_df.copy()
    data_df[['visit_port','ob_discharge_port']] = data_df['current_trip'].str.split('-', expand=True)
    data_df['planned_destination'] = data_df['next_trip'].str.split('-', expand=True)[1]
    data_df['TEUs_evacuated'] = data_df['TEUs_evacuated']
    data_df["timestamp"] = now
    data_df["plan_type"] = ""
    data_df_main = data_df[['rake_name','visit_port','ob_discharge_port','planned_destination','TEUs_evacuated','pendency',
                          'plan_type','currtrp_ETA','rake_status','timestamp', 'current_trip', 'next_trip', 'base_depot_code', 'TXR_Due_Date',
                          'TXR_KmsRem_CurrDest', 'loaded_TEUs_evacuated', 'pendency_loaded', 'empty_TEUs_evacuated', 'pendency_empties']]

    data_df_main[['TEUs_evacuated','pendency',
                  'TXR_KmsRem_CurrDest',
                  'loaded_TEUs_evacuated',
                  'pendency_loaded',
                  'empty_TEUs_evacuated', 'pendency_empties']] = data_df_main[['TEUs_evacuated','pendency','TXR_KmsRem_CurrDest',
                                                                             'loaded_TEUs_evacuated', 'pendency_loaded',
                                                                             'empty_TEUs_evacuated', 'pendency_empties']].fillna(-99).astype(int)

    data_df_main[['rake_name','visit_port','ob_discharge_port','planned_destination']] = data_df_main[['rake_name','visit_port',
                                                                                                     'ob_discharge_port','planned_destination']].astype(str)
    data_df_main.rename(columns = {"timestamp":"load_ts","currtrp_ETA":"ETA"},inplace=True)
    data_df_main['load_ts'] = pd.to_datetime(data_df_main['load_ts'])
    data_df_main['ETA'] = pd.to_datetime(data_df_main['ETA'])
    data_df_main['TXR_Due_Date'] = pd.to_datetime(data_df_main['TXR_Due_Date']).dt.date.astype(str)
    # 'rake_name', 'rake_status', 'current_trip', 'next_trip', 'TEUs_evacuated', 'pendency', 'currtrp_ETA', 'base_depot_code', 'TXR_Due_Date', 'TXR_KmsRem_CurrDest', 'loaded_TEUs_evacuated', 'pendency_loaded', 'empty_TEUs_evacuated', 'pendency_empties'

    print("***************LOAD BQ Table******************")

    df_load_bq = data_df_main[['rake_name','visit_port','ob_discharge_port','planned_destination','TEUs_evacuated','pendency','plan_type','ETA','rake_status','load_ts','TXR_KmsRem_CurrDest','loaded_TEUs_evacuated','pendency_loaded','empty_TEUs_evacuated','pendency_empties','current_trip','next_trip','base_depot_code','TXR_Due_Date']]
    schema=[{"name": "rake_name","type": "STRING"},{"name": "visit_port","type": "STRING"},{"name": "ob_discharge_port","type": "STRING"},{"name": "planned_destination","type": "STRING"},{"name": "TEUs_evacuated","type": "INTEGER"},{"name": "pendency","type": "INTEGER"},{"name": "plan_type","type": "STRING"},{"name": "eta","type": "TIMESTAMP"},{"name": "rake_status","type": "STRING"},{"name": "load_ts","type": "TIMESTAMP"},{"name": "TXR_KmsRem_CurrDest","type": "INTEGER"},{"name": "loaded_TEUs_evacuated","type": "INTEGER"},{"name": "pendency_loaded","type": "INTEGER"},{"name": "empty_TEUs_evacuated","type": "INTEGER"},{"name": "pendency_empties","type": "INTEGER"},{"name": "current_trip","type": "STRING"},{"name": "next_trip","type": "STRING"},{"name": "base_depot_code","type": "STRING"},{"name": "TXR_Due_Date","type": "STRING"}]

    df_load_bq.columns=df_load_bq.columns.str.lower()
    df_load_bq.to_gbq('apsez-svc-prod-datalake.logistics_cleansed.layer2_route_optimization_plan',project_id='apsez-svc-prod-datalake',table_schema=schema,if_exists='append')
    print("loaded {0} rows to {1} ".format(df_load_bq.shape[0],'logistics_cleansed.layer2_route_optimization_plan'))

    print("************ Email Notification *********************")

    schedule_mail_analysis.rename(columns={"currtrp_ETA":"Current_Trip_ETA"})
    schedule_mail_analysis.columns = schedule_mail_analysis.columns.str.upper().str.replace("_"," ")

    def trigger_dag_gcf(data, context=None):

      print(f"event data - {data}")
      web_server_url = (
          "https://2213b29f81334c26ad603a071a655969-dot-asia-south1.composer.googleusercontent.com"
      )
      # Replace with the ID of the DAG that you want to run.
      dag_id = 'email_alert_rail_route_optimizer'

      print(composer2_airflow_rest_api.trigger_dag(web_server_url, dag_id, data))

    #Email_alert([],[],[],schedule_mail_analysis)

    trigger_dag_gcf(cloud_event.data)

    print(schedule_mail_analysis)
    return "done"

TabError: inconsistent use of tabs and spaces in indentation (<ipython-input-2-391f523d4bd4>, line 514)